In [1]:
from importlib import reload
# import os
import pandas as pd
import numpy as np
# import random as rd
import itertools
from datetime import datetime as dt

# from pathlib import Path
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
# from pprint import pprint
import seaborn as sns
# import statsmodels.api as sma
from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import joblib

import sqlite_io as sio
import add_indicators as indic
import split_merge as sm
import balance
import lgbm_mngr
# import model_mngr as modmgr

# reload(modmgr)
reload(balance)
reload(lgbm_mngr)

<module 'lgbm_mngr' from 'G:\\Python\\MarketDataEnrichment\\dataset_mngr\\lgbm_mngr.py'>

In [2]:
# PATH_DATA = "C:\\Projets\\Data"
PATH_DATA = "G:\\Python\\Data"
PATH_DB_FWK=PATH_DATA+"\\sqlite\\dataset_market.db"
PATH_DB_STOCK=PATH_DATA+"\\sqlite\\dataset_paris_stock_adjusted.db"
PATH_DATA_DTS=PATH_DATA+"\\DTS_FULL\\"
PATH_MODEL=PATH_DATA+"\\Models\\LGBM\\"

CONNECTION TO SQLITE

In [3]:
if "con_stock" in locals():
        sio.close_connection(con_stock)
con_stock = sio.get_connection(str_db_path=PATH_DB_STOCK)

if "con_fwk" in locals():
        sio.close_connection(con_fwk)
con_fwk = sio.get_connection(str_db_path=PATH_DB_FWK)

my_session_maker = sessionmaker(bind=con_fwk)
session=my_session_maker()

table_stock="DS_PARIS_1D_CLEAN"

GET DATA

In [7]:
dts_name="PARIS_TREND_1D_20D_V3"
multi_symbol="PARIS_STOCK"

df_base=sio.get_candles_to_df(session=session,con=con_stock, target_table=table_stock,tradable=True)
print(df_base[0:3])


DEBUG: SELECT CODE,OPEN_DATETIME, OPEN,HIGH,LOW,CLOSE,VOLUME FROM DS_PARIS_1D_CLEAN can WHERE can.TIMEFRAME=1440 AND can.SK_SYMBOL IN (SELECT SK_SYMBOL FROM SYMBOL WHERE TRADABLE=1)   
                       OPEN  HIGH   LOW  CLOSE  VOLUME
CODE    OPEN_DATETIME                                 
ABCA.PA 2000-01-03     9.87  9.87  9.87   9.87    3490
        2000-01-04     9.82  9.82  9.82   9.82    4250
        2000-01-05     9.98  9.98  9.98   9.98    5960


In [ ]:
df_work=pd.DataFrame()
for code_value in df_base.index.get_level_values('CODE').unique():
    sub_df=df_base[df_base.index.get_level_values('CODE') == code_value]
    df_work_tmp = indic.add_indicators_to_df(con=con_fwk, df_in=sub_df, dts_name=dts_name,symbol=multi_symbol)
    df_work = pd.concat([df_work, df_work_tmp])
    
df_work.sort_index(inplace=True)
df_work.info() 

In [ ]:
# print(df_base.index.get_level_values('CODE').unique())
# df_work[10000:10010]
# pd.set_option('display.max_columns', None)
# print(df_work.describe())

# save df_work in pkl
df_work.to_pickle(PATH_DATA_DTS+dts_name+"B_BASE.pkl")

 FROM HERE TO LOAD THE BASE FILE

In [ ]:
dts_name="PARIS_TREND_1D_20D_V3"
# dts_name="PARIS_TREND_1D_50D_V1"
multi_symbol="PARIS_STOCK"

# load df_work from pkl
df_work=pd.read_pickle(PATH_DATA_DTS+dts_name+"B_BASE.pkl")

df_work.head(5)

OPEN   HIGH    LOW  CLOSE  VOLUME  sma20  pos_sma20  \
CODE  OPEN_DATETIME                                                         
AB.PA 2010-04-26     12.98  12.98  12.20  12.68   62866    NaN        NaN   
      2010-04-27     12.74  12.83  12.61  12.70   22370    NaN        NaN   
      2010-04-28     12.70  12.70  12.41  12.50    8211    NaN        NaN   
      2010-04-29     12.60  12.65  12.46  12.64    4676    NaN        NaN   
      2010-04-30     12.63  12.71  12.55  12.65    4470    NaN        NaN   

                     sma50  sma200  pos_sma50  ...  perf_atr14_5d  \
CODE  OPEN_DATETIME                            ...                  
AB.PA 2010-04-26       NaN     NaN        NaN  ...            NaN   
      2010-04-27       NaN     NaN        NaN  ...            NaN   
      2010-04-28       NaN     NaN        NaN  ...            NaN   
      2010-04-29       NaN     NaN        NaN  ...            NaN   
      2010-04-30       NaN     NaN        NaN  ...            NaN   

                     perf_adx14_5d  perf_adx14_neg_5d  perf_adx14_pos_5d  \
CODE  OPEN_DATETIME                                                        
AB.PA 2010-04-26               NaN                NaN                NaN   
      2010-04-27               NaN                NaN                NaN   
      2010-04-28               NaN                NaN                NaN   
      2010-04-29               NaN                NaN                NaN   
      2010-04-30               NaN                NaN                NaN   

                     perf_adx14_dif_5d  avg_vol50  pos_avg_vol50  \
CODE  OPEN_DATETIME                                                
AB.PA 2010-04-26                   NaN        NaN            NaN   
      2010-04-27                   NaN        NaN            NaN   
      2010-04-28                   NaN        NaN            NaN   
      2010-04-29                   NaN        NaN            NaN   
      2010-04-30                   NaN        NaN            NaN   

                     perf_williamsr_14_5d  williamsr_14_perf_10d  trix12  
CODE  OPEN_DATETIME                                                       
AB.PA 2010-04-26                      NaN                    NaN     NaN  
      2010-04-27                      NaN                    NaN     NaN  
      2010-04-28                      NaN                    NaN     NaN  
      2010-04-29                      NaN                    NaN     NaN  
      2010-04-30                      NaN                    NaN     NaN  

[5 rows x 160 columns]

Use same dataset than Adjusted (Train + Val + Conf)

In [4]:
name_ds_adj="PARIS_TREND_1D_V5_class_"
df_adj_train=pd.read_pickle(PATH_DATA_DTS+name_ds_adj+"TRAIN.pkl")
df_adj_val= pd.read_pickle(PATH_DATA_DTS+name_ds_adj+"VAL.pkl")
df_adj_conf=pd.read_pickle(PATH_DATA_DTS+name_ds_adj+"CONF.pkl")

df_adj_train['source'] = 'train'
df_adj_val['source'] = 'val'
df_adj_conf['source'] = 'conf'
# concat the 3 dfs
df_adj = pd.concat([df_adj_train, df_adj_val, df_adj_conf], axis=0)
df_adj.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 671570 entries, (Timestamp('1991-10-23 00:00:00'), 'BOI.PA') to (Timestamp('2023-11-01 00:00:00'), 'XFAB.PA')
Columns: 128 entries, OPEN to source
dtypes: float64(127), object(1)
memory usage: 658.7+ MB


In [ ]:
# if  a column is not in df_adj remove from df_work
for col in df_work.columns:
    if col not in df_adj.columns:
        print("remove column",col)
        df_work.drop(col, axis=1, inplace=True)
df_work.info()

In [16]:
# print type of index of df_work and df_adj, there are multi index 2 values
print(f"df_work index type {type(df_work.index.get_level_values('CODE'))} -- df_adj index type {type(df_work.index.get_level_values('OPEN_DATETIME'))}")
print(f"df_work index type {type(df_adj.index.get_level_values('CODE'))} -- df_adj index type {type(df_adj.index.get_level_values('OPEN_DATETIME'))}")
# convert OPEN_DATETIME to a DatetimeIndex
df_work.index = pd.MultiIndex.from_arrays([df_work.index.get_level_values('CODE'), pd.to_datetime(df_work.index.get_level_values('OPEN_DATETIME'))], names=['CODE', 'OPEN_DATETIME'])
print(f"df_work index type {type(df_work.index.get_level_values('CODE'))} -- df_adj index type {type(df_work.index.get_level_values('OPEN_DATETIME'))}")


df_work index type <class 'pandas.core.indexes.base.Index'> -- df_adj index type <class 'pandas.core.indexes.base.Index'>
df_work index type <class 'pandas.core.indexes.base.Index'> -- df_adj index type <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
df_work index type <class 'pandas.core.indexes.base.Index'> -- df_adj index type <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [17]:
# for each code of df_adj, remove the lines from df_work that are before the first date for this code in df_adj
for code_value in df_adj.index.get_level_values('CODE').unique():
    sub_df = df_adj[df_adj.index.get_level_values('CODE') == code_value]
    # get the first date of the code in df_adj
    first_date = sub_df.index.get_level_values('OPEN_DATETIME')[0]
    # remove the lines from df_work that are before the first date for this code in df_adj
    mask = ~(
        (df_work.index.get_level_values('CODE') == code_value) &
        (df_work.index.get_level_values('OPEN_DATETIME') < first_date)
    )
    df_work = df_work[mask]
df_work.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1214241 entries, ('AB.PA', Timestamp('2011-06-08 00:00:00')) to ('XIL.PA', Timestamp('2024-01-15 00:00:00'))
Columns: 115 entries, OPEN to trix12
dtypes: float64(114), int64(1)
memory usage: 1.0+ GB


In [18]:
# remove lines if sma20_200<0
df_work = df_work[df_work['pos_sma20_200'] >= 0]
df_work.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 662333 entries, ('AB.PA', Timestamp('2011-06-08 00:00:00')) to ('XIL.PA', Timestamp('2022-09-09 00:00:00'))
Columns: 115 entries, OPEN to trix12
dtypes: float64(114), int64(1)
memory usage: 584.0+ MB


In [ ]:
dts_name="PARIS_TREND_1D_20D_V3"

df_work.to_pickle(PATH_DATA_DTS+dts_name+"B_1_BASE.pkl")

df_work = pd.read_pickle(PATH_DATA_DTS+dts_name+"B_1_BASE.pkl")

In [6]:
#  list columns in df_adj and not in df_work
list_col=[]
for col in df_adj.columns:
    if col not in df_work.columns:
        list_col.append(col)
print("list of columns in df_adj and not in df_work",list_col)
# print col from df_work named like 'lab_perf%'
list_col=[]
for col in df_work.columns:
    if col.startswith('lab_perf'):
        list_col.append(col)
print("list of columns in df_work starting with 'lab_perf'",list_col)


list of columns in df_adj and not in df_work ['lab_perf_5d_class_3', 'lab_perf_5d_class_5', 'lab_perf_5d_class_10', 'lab_perf_10d_class_3', 'lab_perf_10d_class_5', 'lab_perf_10d_class_10', 'lab_perf_20d_class_3', 'lab_perf_20d_class_5', 'lab_perf_20d_class_10', 'lab_perf_50d_class_3', 'lab_perf_50d_class_5', 'lab_perf_50d_class_10', 'source']
list of columns in df_work starting with 'lab_perf' ['lab_perf_20d', 'lab_perf_50d', 'lab_perf_10d', 'lab_perf_5d']


In [9]:
#  print min and max open datetime for each source of df_adj
# Reset index to access OPEN_DATETIME as a column if it's in the index
df_adj_reset = df_adj.reset_index()
source_ranges = df_adj_reset.groupby('source')['OPEN_DATETIME'].agg(['min', 'max']).to_dict('index')
print(source_ranges)

def assign_source(row):
    dt = row.name[1] if isinstance(row.name, tuple) else row['OPEN_DATETIME']
    for source, rng in source_ranges.items():
        if rng['min'] <= dt <= rng['max']:
            return source
    return None

# Step 3: Apply to df_work (assuming MultiIndex with ['CODE', 'OPEN_DATETIME'])
df_work['source'] = df_work.apply(assign_source, axis=1)

df_work_reset = df_work.reset_index()
source_ranges = df_work_reset.groupby('source')['OPEN_DATETIME'].agg(['min', 'max']).to_dict('index')
print(source_ranges)

{'conf': {'min': Timestamp('2020-10-01 00:00:00'), 'max': Timestamp('2023-11-01 00:00:00')}, 'train': {'min': Timestamp('1991-10-23 00:00:00'), 'max': Timestamp('2017-07-31 00:00:00')}, 'val': {'min': Timestamp('2017-08-01 00:00:00'), 'max': Timestamp('2020-09-30 00:00:00')}}
{'conf': {'min': Timestamp('2020-10-01 00:00:00'), 'max': Timestamp('2023-11-01 00:00:00')}, 'train': {'min': Timestamp('1992-01-28 00:00:00'), 'max': Timestamp('2017-07-31 00:00:00')}, 'val': {'min': Timestamp('2017-08-01 00:00:00'), 'max': Timestamp('2020-09-30 00:00:00')}}


In [3]:
dts_name="PARIS_TREND_1D_20D_V3"

# df_work.to_pickle(PATH_DATA_DTS+dts_name+"B_2_BASE.pkl")

df_work = pd.read_pickle(PATH_DATA_DTS+dts_name+"B_2_BASE.pkl")

Add class labels

In [4]:
label = "lab_perf_20d"

# add a 'split' column to df_work with The values must be 0=train , 1=validation, 2=confirmation frm 'source'
df_work['split'] = df_work['source'].map({'train': 0, 'val': 1, 'conf': 2}).astype('Int64')

df_split=sm.split_df_by_split_value(df_in=df_work, column_name= "split") 
   
df_class = df_split[0]
df_class_val = df_split[1]
df_class_conf = df_split[2]
df_class.sort_index(inplace=True)
df_class_val.sort_index(inplace=True)
df_class_conf.sort_index(inplace=True)

print(f"selected: {df_class.shape=} valid: {df_class_val.shape=} confirm: {df_class_conf.shape=}")


C:\Users\Benoit\AppData\Local\Temp\ipykernel_2588\4035861643.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_work['split'] = df_work['source'].map({'train': 0, 'val': 1, 'conf': 2}).astype('Int64')


selected: df_class.shape=(449071, 116) valid: df_class_val.shape=(95313, 116) confirm: df_class_conf.shape=(113630, 116)


In [ ]:
label = "lab_perf_50d"
nb_class=10
df_class=balance.add_class_by_lab_nb_lines(df_in=df_class,str_label=label,nb_class=nb_class,bool_replace_label=False)
min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
print(min_max_lab_by_class)

                         min       max
lab_perf_50d_class                    
0                  -0.904382 -0.118562
1                  -0.118561 -0.064935
2                  -0.064935 -0.031787
3                  -0.031787 -0.004444
4                  -0.004444  0.020442
5                   0.020443  0.047537
6                   0.047537  0.078117
7                   0.078119  0.117239
8                   0.117241  0.182157
9                   0.182158  9.458328


In [ ]:
label = "lab_perf_50d"
nb_class=10
# categ_5={0:[-1,-0.009992],1:[-0.009992,0.013378],2:[0.013378,10]}
# categ_5={0:[-1,-0.023155],1:[-0.023155,-0.004950],2:[-0.004950,0.007881],3:[0.007881,0.028120],4:[0.028120,10]}
# categ_5={0:[-1,-0.040363],1:[-0.040363,-0.023155],2:[-0.023155,-0.012785],3:[-0.012785,-0.004950],4:[-0.004950,0.000667],
        #  5:[0.000667,0.007881],6:[0.007881,0.016478],7:[0.016478,0.028120],8:[0.028120,0.048214],9:[0.048214,10]}
# categ_10={0:[-1,-0.013699],1:[-0.013699,0.021450],2:[0.021450,10]}
# categ_10={0:[-1,-0.032550],1:[-0.032550,-0.006397],2:[-0.006397,0.013293],3:[0.013293,0.042789],4:[0.042789,10]}
# categ_10={0:[-1,-0.056258],1:[-0.056258,-0.032550],2:[-0.032550,-0.017791],3:[-0.017791,-0.006397],4:[-0.006397,0.002747],
        # 5:[0.002747,0.013293],6:[0.013293,0.025992],7:[0.025992,0.042789],8:[0.042789,0.070477],9:[0.070477,10]}
# categ_20={0:[-1,-0.018090],1:[-0.018090,0.034697],2:[0.034697,10]}
# categ_20={0:[-1,-0.044845],1:[-0.044845,-0.007652],2:[-0.007652,0.022616],3:[0.022616,0.065744],4:[0.065744,10]}
# categ_20={0:[-1,-0.077964],1:[-0.077964,-0.044845],2:[-0.044845,-0.023898],3:[-0.023898,-0.007652],4:[-0.007652,0.006682],
        # 5:[0.006682,0.022616],6:[0.022616,0.041398],7:[0.041398,0.065744],8:[0.065744,0.105263],9:[0.105263,10]}
# categ_50={0:[-1,-0.022242],1:[-0.022242,0.067331],2:[0.067331,10]}
# categ_50={0:[-1,-0.064935],1:[-0.064935,-0.004444],2:[-0.004444,0.047537],3:[0.047537,0.117239],4:[0.117239,10]}
# categ_50={0:[-1,-0.118562],1:[-0.118562,-0.064935],2:[-0.064935,-0.031787],3:[-0.031787,-0.004444],4:[-0.004444,0.020442],
        # 5:[0.020442,0.047537],6:[0.047537,0.078117],7:[0.078117,0.117239],8:[0.117239,0.182157],9:[0.182157,10]}

suffix_class='_class_'+str(nb_class)
df_class=balance.add_lab_by_class(df_in=df_class,str_label=label, categ=categ_50,
                                  bool_replace_label=False,str_suffix_class=suffix_class) # categ
df_class.sort_index(inplace=True)
df_class_val=balance.add_lab_by_class(df_in=df_class_val,str_label=label, categ=categ_50,
                                      bool_replace_label=False,str_suffix_class=suffix_class) # categ
df_class_val.sort_index(inplace=True)
df_class_conf=balance.add_lab_by_class(df_in=df_class_conf,str_label=label, categ=categ_50,
                                       bool_replace_label=False,str_suffix_class=suffix_class) # categ
df_class_conf.sort_index(inplace=True)

new_lab=label+suffix_class

print(df_class.loc[:, new_lab].dropna().iloc[[0, -1]])
print(df_class_val.loc[:, new_lab].dropna().iloc[[0, -1]])
print(df_class_conf.loc[:, new_lab].dropna().iloc[[0, -1]])

data = df_class[new_lab]
print(data.value_counts().sort_index())
data_val = df_class_val[new_lab]
print(data_val.value_counts().sort_index())
data_conf = df_class_conf[new_lab]
print(data_conf.value_counts().sort_index())


CODE    OPEN_DATETIME
AB.PA   2011-06-08       0.0
XIL.PA  2017-07-31       7.0
Name: lab_perf_50d_class10, dtype: float64
CODE    OPEN_DATETIME
AB.PA   2019-04-16       1.0
XIL.PA  2020-09-30       3.0
Name: lab_perf_50d_class10, dtype: float64
CODE    OPEN_DATETIME
AB.PA   2020-10-01       1.0
XIL.PA  2022-09-09       0.0
Name: lab_perf_50d_class10, dtype: float64
lab_perf_50d_class10
0.0    44907
1.0    44914
2.0    44901
3.0    44908
4.0    44905
5.0    44909
6.0    44905
7.0    44907
8.0    44907
9.0    44908
Name: count, dtype: int64
lab_perf_50d_class10
0.0    17017
1.0    10147
2.0     9647
3.0     9807
4.0     9480
5.0     9342
6.0     8371
7.0     7299
8.0     6655
9.0     7535
Name: count, dtype: int64
lab_perf_50d_class10
0.0    15643
1.0    13344
2.0    12163
3.0    11261
4.0    10036
5.0     9987
6.0     9763
7.0     9594
8.0     9960
9.0    11779
Name: count, dtype: int64


START HERE !

In [20]:
# clean V4 dataset to make a V5

new_dts_name="PARIS_TREND_1D_V5_class"

df_class_v5=df_class.copy()
df_class_val_v5=df_class_val.copy()
df_class_conf_v5=df_class_conf.copy()

#add column CODE to the index to get a multi index ALREADY IN INDEX
# df_class_v5=df_class_v5.set_index('CODE',append=True)
# df_class_val_v5=df_class_val_v5.set_index('CODE',append=True)
# df_class_conf_v5=df_class_conf_v5.set_index('CODE',append=True)

# drop col stdev20_sma20 and stdev20_sma5
df_class_v5.drop(columns=['stdev20_sma20','stdev20_sma5'],inplace=True,errors='ignore')
df_class_val_v5.drop(columns=['stdev20_sma20','stdev20_sma5'],inplace=True,errors='ignore')
df_class_conf_v5.drop(columns=['stdev20_sma20','stdev20_sma5'],inplace=True,errors='ignore')

# delete ALVDM.PA before 2017
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "ALVDM.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2017-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "ALVDM.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2017-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "ALVDM.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2017-01-01")].index,inplace=True)

# delete SAMS.PA LOUP.PA ELEC.PA CRSU.PA IMDA.PA LOCAL.PA EUR.PA ALFLE.PA BUR.PA
list_code_to_del=["SAMS.PA","LOUP.PA","ELEC.PA","CRSU.PA","IMDA.PA","LOCAL.PA","EUR.PA","ALFLE.PA","BUR.PA"]
for code in list_code_to_del:
    df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == code)].index,inplace=True)
    df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == code)].index,inplace=True)
    df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == code)].index,inplace=True)

# delete ALGIL.PA before 2012
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "ALGIL.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2012-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "ALGIL.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2012-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "ALGIL.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2012-01-01")].index,inplace=True)

# delete EIFF.PA before 2007
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "EIFF.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2007-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "EIFF.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2007-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "EIFF.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2007-01-01")].index,inplace=True)

# delete ALMDT.PA before 2020
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "ALMDT.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "2020-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "ALMDT.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "2020-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "ALMDT.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "2020-01-01")].index,inplace=True)

# delete GFC.PA before 1998
df_class_v5.drop(df_class_v5[(df_class_v5.index.get_level_values('CODE') == "GFC.PA") & 
                             (df_class_v5.index.get_level_values('OPEN_DATETIME') < "1998-01-01")].index,inplace=True)
df_class_val_v5.drop(df_class_val_v5[(df_class_val_v5.index.get_level_values('CODE') == "GFC.PA") &
                                    (df_class_val_v5.index.get_level_values('OPEN_DATETIME') < "1998-01-01")].index,inplace=True)
df_class_conf_v5.drop(df_class_conf_v5[(df_class_conf_v5.index.get_level_values('CODE') == "GFC.PA") &
                                      (df_class_conf_v5.index.get_level_values('OPEN_DATETIME') < "1998-01-01")].index,inplace=True)

print(f"{df_class_v5.shape=} {df_class_val_v5.shape=} {df_class_conf_v5.shape=}")

df_class_v5.to_pickle(PATH_DATA_DTS+new_dts_name+"_TRAIN.pkl")
df_class_val_v5.to_pickle(PATH_DATA_DTS+new_dts_name+"_VAL.pkl")
df_class_conf_v5.to_pickle(PATH_DATA_DTS+new_dts_name+"_CONF.pkl")



df_class_v5.shape=(425535, 128) df_class_val_v5.shape=(91867, 128) df_class_conf_v5.shape=(110839, 128)


In [21]:
dts_name="PARIS_TREND_1D_V5_class"
print(df_class_v5.head(5))
# print list of columns of df_class in a txt file
with open(PATH_DATA_DTS+dts_name+"_class_TRAIN_columns.txt", "w") as output:
    output.write(str(df_class_v5.columns.values.tolist()))

                      OPEN   HIGH    LOW  CLOSE  VOLUME  pos_sma20  pos_sma50  \
CODE  OPEN_DATETIME                                                             
AB.PA 2011-06-08     10.77  10.95  10.01  10.12  182867   0.082642   0.096401   
      2011-06-09     10.15  10.27   9.74  10.04  132862   0.065874   0.086157   
      2011-06-10     10.11  10.14   9.91  10.00   51809   0.055242   0.080497   
      2011-06-13     10.15  10.65  10.08  10.60  211391   0.110180   0.142438   
      2011-06-14     10.62  11.23  10.45  10.66  302799   0.107763   0.145498   

                     pos_sma200  pos_sma50_200  pos_sma20_50  ...  \
CODE  OPEN_DATETIME                                           ...   
AB.PA 2011-06-08       0.085261      -0.010161      0.012708  ...   
      2011-06-09       0.077629      -0.007851      0.019029  ...   
      2011-06-10       0.074316      -0.005721      0.023933  ...   
      2011-06-13       0.139479      -0.002591      0.029057  ...   
      2011-06-14  

In [10]:
df_class=df_class_v5.copy()
df_class_val=df_class_val_v5.copy()
df_class_conf=df_class_conf_v5.copy()

In [22]:
# ROUND THE DATASETS
# round_to_significant_digits(df : pd.DataFrame, digits : int, list_col_int:list=None, list_col_ignore:list=None)
pd.set_option('display.max_columns', None)

digits=3
list_col_int=["rsi14","sma5_rsi14","sma20_rsi14","aroon14_up","aroon14_down","aroon14_dif",
"stoch14","stoch14_signal","stoch14_dif","adx14","adx14_neg","adx14_pos",
"perf_rsi14_5d","perf_rsi14_10d","aroon14_up_lag_5d","aroon14_down_lag_5d",
"aroon14_dif_lag_5d",	"perf_stoch14_5d","perf_stoch14_signal_5d",
"perf_stoch14_dif_5d","perf_adx14_neg_5d",	"perf_adx14_pos_5d",	
"perf_williamsr_14_5d","williamsr_14_perf_10d","williamsr_14"	]

list_col_ignore=["CODE","OPEN", "HIGH", "LOW", "CLOSE", "VOLUME","lab_perf_20d","lab_perf_50d","lab_perf_125d",
                  "lab_perf_10d","lab_perf_5d", "lab_perf_5d_class_3",
 "lab_perf_5d_class_5", "lab_perf_5d_class_10", "lab_perf_10d_class_3", "lab_perf_10d_class_5",
 "lab_perf_10d_class_10", "lab_perf_20d_class_3", "lab_perf_20d_class_5", "lab_perf_20d_class_10",
 "lab_perf_50d_class_3", "lab_perf_50d_class_5", "lab_perf_50d_class_10"]

df_class_r=balance.round_to_significant_digits(df=df_class,digits=digits,
                                               list_col_int=list_col_int,list_col_ignore=list_col_ignore)
df_class_val_r=balance.round_to_significant_digits(df=df_class_val,digits=digits,
                                               list_col_int=list_col_int,list_col_ignore=list_col_ignore)
df_class_conf_r=balance.round_to_significant_digits(df=df_class_conf,digits=digits,
                                               list_col_int=list_col_int,list_col_ignore=list_col_ignore)


df_class_r.to_pickle(PATH_DATA_DTS+dts_name+"_R"+str(digits)+"_TRAIN.pkl")
df_class_val_r.to_pickle(PATH_DATA_DTS+dts_name+"_R"+str(digits)+"_VAL.pkl")
df_class_conf_r.to_pickle(PATH_DATA_DTS+dts_name+"_R"+str(digits)+"_CONF.pkl")


In [ ]:
# read the 3 pkl files
dts_name="PARIS_TREND_1D_V5_class_R3"

df_class_r=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val_r=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_class_conf_r=pd.read_pickle(PATH_DATA_DTS+dts_name+"_CONF.pkl")


In [23]:
pd.set_option('display.max_columns', None)
print(f"{df_class_r.shape=} {df_class_val_r.shape=} ")
print(f"{df_class_r.describe()}")
print(f"{df_class_r[1000:1050]}")


df_class_r.shape=(449071, 128) df_class_val_r.shape=(95313, 128) 


g:\Python\MarketDataEnrichment\.env\lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


                OPEN           HIGH            LOW          CLOSE  \
count  449071.000000  449071.000000  449071.000000  449071.000000   
mean       50.318183      50.828004      49.781571      50.335887   
std       148.342842     150.015082     146.643742     148.398788   
min         0.112400       0.112400       0.112400       0.410000   
25%        14.010000      14.190000      13.875000      14.030000   
50%        27.660000      27.940000      27.355000      27.675000   
75%        53.326450      53.900000      52.790000      53.350000   
max      3810.911900    3834.253400    3776.677500    3820.248500   

             VOLUME      pos_sma20      pos_sma50     pos_sma200  \
count  4.490710e+05  449071.000000  449071.000000  449071.000000   
mean   4.089459e+05       0.009218       0.033331       0.137672   
std    1.326412e+06       0.049215       0.078822       0.150711   
min    0.000000e+00      -0.772000      -0.748000      -0.632000   
25%    2.005000e+03      -0.015000    

In [ ]:
# Clip of the features 
feature_limits = {
    'pos_sma20': [-0.5, 0.5],
    'pos_sma50': [-0.5, 1],
    'pos_sma200': [-0.5, 1],
    'pos_sma50_200': [-0.25, 1],
    'ret_1d': [-0.25, 0.25],
    'ret_5d': [-0.5, 0.5],
    'stdev20_1d': [None, 5],
    'pos_stdev20_sma20': [None, 4],
    'macd_dif': [-5, 5],
    'pos_stdev20_sma5': [None, 1],
    'pos_ema50_perf_5d': [-10, 10],
    'pos_ema50_perf_10d': [-10, 10],
    'pos_bb20_lo_lag_5d': [None, 2],
    'pos_bot20_lag_5d': [None, 1],
    'pos_bot50_lag_10d': [None, 1],
    'perf_atr14_5d': [None, 2],
    'perf_adx14_5d': [None, 2],
    'pos_avg_vol50': [None, 10],
    'pos_bot20': [None, 1],
    'pos_bot50': [None, 1],
    'pos_bot_200': [None, 4],
    'pos_rsi14_sma5': [-0.75, 0.75],
    'pos_rsi14_sma20': [-0.75, 0.75],
    'pos_rsi14_sma5_20': [-0.75, 0.75],
    'pos_sma20_200': [None, 1],
    'pos_sma10': [-0.5, 0.5],
    'pos_sma5': [-0.25, 0.25],
    'perf_pos_sma10_10d': [-0.5, 0.5],
    'perf_sma10_5d': [-0.25, 0.25],
    'perf_sma10_10d': [-0.5, 0.5],
    'perf_pos_sma20_10d': [-0.5, 0.5],
    'perf_sma20_10d': [-0.25, 0.25],
    'perf_pos_sma50_10d': [-0.5, 0.5],
    'perf_pos_sma200_10d': [-0.5, 0.5],
    'pos_sma5_50': [-0.5, 0.5],
    'pos_sma5_200': [-0.5, 1],
    'pos_sma10_50': [-0.5, 0.5],
    'pos_sma10_200': [-0.25, 1],
    'perf_ema5_5d': [-0.25, 0.25],
    'pos_ema10': [-0.25, 0.25],
    'perf_pos_ema10_5d': [-0.25, 0.25],
    'perf_pos_ema10_10d': [-0.25, 0.25],
    'perf_pos_ema20_5d': [-0.25, 0.25],
    'perf_pos_ema20_10d': [-0.5, 0.5],
    'perf_pos_ema200_5d': [-0.5, 0.5],
    'perf_pos_ema200_10d': [-0.5, 0.5],
    'pos_bb20_lo_lag_5d': [-0.5, 0.5],
    'ret_10d': [-0.5, 0.5],
    'ret_20d': [-0.5, 0.5],
    'pos_bot20_lag_5d': [None, 0.5],
    'pos_bot50_lag_10d': [None, 1],
    'macd_fast': [-5, 5],
    'macd_slow': [-5, 5],
    'perf_atr14_5d': [None, 2]
}

for feature, limits in feature_limits.items():
    min_val, max_val = limits
    try:
        df_class_r = balance.clipping_col(df_class_r, feature, min_val, max_val)
    except KeyError:
        print(f"Feature '{feature}' not found in df_class. Skipping...")
    try:
        df_class_val_r = balance.clipping_col(df_class_val_r, feature, min_val, max_val)
    except KeyError:
        print(f"Feature '{feature}' not found in df_class_val. Skipping...")
    try:
        df_class_conf_r = balance.clipping_col(df_class_conf_r, feature, min_val, max_val)
    except KeyError:
        print(f"Feature '{feature}' not found in df_class_conf. Skipping...")

dts_name="PARIS_TREND_1D_V5B_class_R3"
df_class_r.to_pickle(PATH_DATA_DTS+dts_name+"_CLIP_TRAIN.pkl")
df_class_val_r.to_pickle(PATH_DATA_DTS+dts_name+"_CLIP_VAL.pkl")
df_class_conf_r.to_pickle(PATH_DATA_DTS+dts_name+"_CLIP_CONF.pkl")

In [ ]:
list_col=['pos_sma20','pos_sma50','pos_sma200','pos_sma50_200','ret_1d','ret_5d'
,'stdev20_1d','pos_stdev20_sma20','macd_dif','pos_stdev20_sma5'
,'pos_ema50_perf_5d','pos_ema50_perf_10d','pos_bb20_lo_lag_5d','pos_bot20_lag_5d','pos_bot50_lag_10d'
,'perf_atr14_5d','perf_adx14_5d','pos_avg_vol50','pos_bot20','pos_bot50','pos_bot_200'
,'pos_rsi14_sma5','pos_rsi14_sma20','pos_rsi14_sma5_20','pos_sma20_200','pos_sma10'
,'pos_sma5','perf_pos_sma10_10d','perf_sma10_5d','perf_sma10_10d'  ,'perf_pos_sma20_10d'
,'perf_sma20_10d'  ,'perf_pos_sma50_10d','perf_pos_sma200_10d','pos_sma5_50'   
,'pos_sma5_200','pos_sma10_50' ,'pos_sma10_200','perf_ema5_5d'      ,'pos_ema10'  
,'perf_pos_ema10_5d'  ,'perf_pos_ema10_10d','perf_pos_ema20_5d'  ,'perf_pos_ema20_10d'  
,'perf_pos_ema200_5d'  ,'perf_pos_ema200_10d'
,'pos_bb20_lo_lag_5d','ret_10d'        ,'ret_20d','pos_bot20_lag_5d'  ,'pos_bot50_lag_10d'
,'macd_fast'      ,'macd_slow','perf_atr14_5d']

# for each col in list_col print a graph of the distribution of the values
for col in list_col:
    print(col)
    sns.histplot(data=df_class_r, x=col, kde=True, bins=50, color='blue')
    plt.show()

In [3]:
dts_name="PARIS_TREND_1D_V5B_class_R3_CLIP"
df_class=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")

print(f"{df_class.shape} {df_class_val.shape}")

(449071, 128) (95313, 128)


In [4]:
# remove useless columns from df_class
# list_rem=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME',
# 'lab_perf_20d','lab_perf_50d','lab_perf_10d','lab_perf_5d', 'lab_perf_5d','CODE',
# 'lab_perf_5d_class_3',
# 'lab_perf_5d_class_5',
# 'lab_perf_5d_class_10',
# 'lab_perf_10d_class_3',
# 'lab_perf_10d_class_5',
# 'lab_perf_10d_class_10',
# 'lab_perf_20d_class_3',
# 'lab_perf_20d_class_5', # the label to predict
# # 'lab_perf_20d_class_10',
# 'lab_perf_50d_class_3',
# 'lab_perf_50d_class_5',
# 'lab_perf_50d_class_10']

list_rem=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME',
'lab_perf_20d','lab_perf_50d','lab_perf_10d','lab_perf_5d', 'lab_perf_5d','CODE',
'lab_perf_5d_class3',
'lab_perf_5d_class5',
'lab_perf_5d_class10',
'lab_perf_10d_class3',
'lab_perf_10d_class5',
'lab_perf_10d_class10',
'lab_perf_20d_class3',
'lab_perf_20d_class5', # the label to predict
'lab_perf_20d_class_10',
'lab_perf_50d_class3',
# 'lab_perf_50d_class5',
'lab_perf_50d_class10'
]


# remove columns from df_x_train_r don't crash if not in list
df_class.drop(columns=list_rem, inplace=True, errors='ignore')
df_class_val.drop(columns=list_rem, inplace=True, errors='ignore')

print(f"{df_class.shape} {df_class_val.shape}")

(449071, 109) (95313, 109)


In [ ]:
label='lab_perf_50d_class_5'
# rename col lab_perf_20d_class10 by label 
df_class.rename(columns={'lab_perf_50d_class5': label}, inplace=True, errors='ignore')
df_class_val.rename(columns={'lab_perf_50d_class5': label}, inplace=True, errors='ignore')

# do a drop na to clean the dataframes
df_class.dropna( inplace=True)
df_class_val.dropna(inplace=True)

print(df_class[label].value_counts().sort_index()) # min 44904  >> 44890 -- 89790
print(df_class_val[label].value_counts().sort_index()) # min 7918 >> 7900 -- 14180

lab_perf_50d_class_5
0.0    89804
1.0    89798
2.0    89800
3.0    89798
4.0    89801
Name: count, dtype: int64
lab_perf_50d_class_5
0.0    27071
1.0    19433
2.0    18810
3.0    15664
4.0    14188
Name: count, dtype: int64


In [6]:
# UNDERSAMPLE THE TRAIN AND VAL AND SAVE DATASET FOR TRAINING
list_feat=df_class.columns.values.tolist()
list_feat.remove(label)

df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)

df_x_val, col_y_val = sm.split_df_x_y(
    df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)

#  for classification datasets
lim_train=89790
lim_val=14180
# method = RandomUnderSampler(sampling_strategy={0:lim_train,1:lim_train,2:lim_train,3:lim_train,4:lim_train,
                                            #    5:lim_train,6:lim_train,7:lim_train,8:lim_train,9:lim_train}) 
method = RandomUnderSampler(sampling_strategy={0:lim_train,1:lim_train,2:lim_train,3:lim_train,4:lim_train})
df_x_train, col_y_train=  method.fit_resample(df_x_train, col_y_train)

# method = RandomUnderSampler(sampling_strategy={0:lim_val,1:lim_val,2:lim_val,3:lim_val,4:lim_val,
#                                                5:lim_val,6:lim_val,7:lim_val,8:lim_val,9:lim_val}) 
method = RandomUnderSampler(sampling_strategy={0:lim_val,1:lim_val,2:lim_val,3:lim_val,4:lim_val}) 
df_x_val, col_y_val=  method.fit_resample(df_x_val, col_y_val)

joblib.dump(df_x_train, PATH_DATA_DTS+dts_name+"_"+label+"_X_TRAIN.pkl")
joblib.dump(col_y_train, PATH_DATA_DTS+dts_name+"_"+label+"_Y_TRAIN.pkl")
joblib.dump(df_x_val, PATH_DATA_DTS+dts_name+"_"+label+"_X_VAL.pkl")
joblib.dump(col_y_val, PATH_DATA_DTS+dts_name+"_"+label+"_Y_VAL.pkl")


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5B_class_R3_CLIP_lab_perf_50d_class_5_Y_VAL.pkl']

In [ ]:
# #ONLY FIRST TIME 

# print(df_x_train.shape)
# print(df_x_val.shape)

# list_rem=['perf_pos_ema20_10d','perf_pos_ema20_5d','perf_stoch14_5d','perf_stoch14_dif_5d'
# ,'perf_williamsr_14_5d','pos_ema10','pos_ema5','pos_sma10','ret_1d','stoch14_dif','perf_pos_ema200_10d'
# ,'pos_avg_vol14','aroon14_down','ret_10d','perf_sma10_5d','perf_adx14_neg_5d','ret_5d'
# ,'perf_sma5_5d','perf_pos_ema200_5d','perf_pos_ema10_5d','perf_sma_10_5d','perf_pos_sma10_10d'
# ,'stdev20_1d','perf_adx14_pos_5d','perf_pos_ema10_10d','perf_stoch14_signal_5d','pos_sma5_10'
# ,'tr_atr14','pos_ema50_perf_5d','pos_ema50_perf_10d','pos_rsi14_sma5','williamsr_14_perf_10d'
# ,'rsi14','perf_pos_sma200_10d','perf_pos_sma50_10d','pos_sma5','aroon14_up_lag_5d'
# ,'pos_rsi14_sma20','pos_sma10_20','perf_rsi14_10d','perf_ema5_5d','aroon14_dif_lag_5d'
# ,'perf_adx14_dif_5d','pos_bb20_hi_lag_5d','perf_pos_sma20_10d','aroon14_dif'
# ,'pos_sma5_50','pos_rsi14_sma5_20','perf_rsi14_5d','sma5_rsi14','pos_sma20'
# ,'stoch14','perf_sma10_10d','pos_sma5_20','pos_bb20_hi','pos_sma10_50','pos_stdev20_sma5'
# ,'perf_sma20_5d','perf_sma50_5d','pos_sma50','aroon14_down_lag_5d','pos_bb20_lo_lag_5d'
# ,'pos_bot20','pos_top20','pos_top20_lag_5d','perf_sma_20_5d','pos_bot20_lag_5d','perf_sma_200_5d','pos_sma50_200'
# ]

# # remove columns from df_x_train_r don't crash if not in list
# df_x_train.drop(columns=list_rem, inplace=True, errors='ignore')
# df_x_val.drop(columns=list_rem, inplace=True, errors='ignore')

# print(df_x_train.shape)
# print(df_x_val.shape)

# # save the 2 df in pkl format
# joblib.dump(df_x_train, PATH_DATA_DTS+dts_name+"_"+label+"_X_TRAIN.pkl")
# joblib.dump(df_x_val, PATH_DATA_DTS+dts_name+"_"+label+"_X_VAL.pkl")


(448900, 107)
(79000, 107)
(448900, 38)
(79000, 38)


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5B_class_R3_CLIP_lab_perf_20d_class_10_X_VAL.pkl']

In [19]:
# drop from df_x_train_r and df_x_val_r the columns pos_sma50_200 and pos_sma20_200
# save list of columns of df_x_train_r in an info file   
nb_digits="3"
with open(PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+nb_digits+"_INFO.txt", "w") as f:
    for s in df_x_train.columns:
        f.write(str(s) +"\n")

In [3]:
# Load datasets before training
dts_name="PARIS_TREND_1D_V5B_class_R3_CLIP"
label="lab_perf_50d_class_5"

df_x_train_r = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_X_TRAIN.pkl")
col_y_train = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_Y_TRAIN.pkl")
df_x_val_r = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_X_VAL.pkl")
col_y_val = joblib.load(PATH_DATA_DTS+dts_name+"_"+label+"_Y_VAL.pkl")

print(f"{df_x_train_r.shape=} {df_x_val_r.shape=} {col_y_train.size=} {col_y_val.size=}")

df_x_train_r.shape=(448950, 108) df_x_val_r.shape=(70900, 108) col_y_train.size=448950 col_y_val.size=70900


In [4]:
list_keep=['pos_bot_200','pos_ema50_200','adx14','macd_dif','pos_sma50_200','pos_top_200','adx14_neg'
,'pos_bot50','adx14_pos','perf_sma200_10d','macd_fast','pos_stdev20_sma20','pos_bot50_lag_10d','perf_sma50_10d'
,'sma20_rsi14','pos_sma20_200','perf_adx14_5d','macd_slow','perf_atr14_5d','pos_sma200','pos_sma20_50','adx14_dif'
,'pos_bot20_lag_20d','perf_sma20_10d','pos_top50','pos_avg_vol50','stoch14_signal','sma5_rsi14','pos_rsi14_sma5_20'
,'pos_sma10_200','pos_top50_lag_10d','pos_bb20_hi_lag_5d','pos_top20_lag_20d','pos_sma5_200','pos_bot20'
,'stoch14','pos_bb20_lo_lag_5d','pos_stdev20_sma5','trix12','aroon14_dif','pos_bb20_hi','rsi14','perf_adx14_neg_5d'
,'tr_atr14','williamsr_14','pos_bb20_lo','pos_sma50','perf_adx14_pos_5d','perf_stoch14_dif_5d','ret_20d'
,'pos_bot20_lag_5d','pos_sma10_50','pos_sma5_50','stdev20_1d','aroon14_dif_lag_5d','pos_top20','perf_sma200_5d'
,'perf_sma_200_5d','perf_sma50_5d']

# remove columns from df_x_train_r idf not in list_keep, don't crash if not in list
df_x_train_r.drop(columns=[col for col in df_x_train_r.columns if col not in list_keep], inplace=True, errors='ignore')
df_x_val_r.drop(columns=[col for col in df_x_val_r.columns if col not in list_keep], inplace=True, errors='ignore')

print(f"{df_x_train_r.shape=} {df_x_val_r.shape=} {col_y_train.size=} {col_y_val.size=}")

df_x_train_r.shape=(448950, 59) df_x_val_r.shape=(70900, 59) col_y_train.size=448950 col_y_val.size=70900


In [ ]:
# print df_x_train_r describe() but with all the columns
pd.set_option('display.max_columns', None)
print(df_x_train_r.describe())

LIGHTGBM

In [5]:
# lgbm_mngr.py 

import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

# Create a scorer object
custom_scorer = metrics.make_scorer(lgbm_mngr.lgbm_custom_penalty_score, is_higher_better=False)

def print_eval_metric(env):
    # env is a callback environment that contains information about the training process
    result = env.evaluation_result_list
    for item in result:
        metric_name, dataset_name, metric_value, is_higher_better = item
        print(f"Iteration {env.iteration}: {dataset_name} - {metric_name} = {metric_value}")
        

In [ ]:
# for classification datasets !!!!!!!!!

device_type='gpu'
results_df = pd.DataFrame(columns=['params','accuracy_train', 'accuracy_val','penalty_score',
                                   'precision','recall','best_iter','seed','max_max_depth','avg_max_depth'])

params = {
    'learning_rate': [0.0001],  # 0.0005 ?
    'max_depth': [20],  # np.linspace(11,15,3, endpoint=True,dtype=int),
    'num_leaves': [23,31,47],  # 1023,1536],#np.linspace(15, 60,10, endpoint=True,dtype=int),
    'min_data_in_leaf': [25,50],
    'feature_fraction': [0.25],
    'lambda_l1': [0],#[0,0.1,1,5] np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
    'lambda_l2': [0.1,0.5,1],#[0,0.1,1,5]  np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
}

nb_class=5
nb_seeds=10
n_splits=3
method='strat'

if method=='time':
    scv = TimeSeriesSplit(n_splits=1)
elif method=='strat':
    scv = StratifiedKFold(n_splits=n_splits, shuffle=False)
else:
    raise ValueError(f"Unknown method: {method}")

# Generate all combinations of parameters
keys, values = zip(*params.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Print the list of parameter combinations
print(f"{combinations.__len__()=}")
for combo in combinations[:2]:  # Print only the first 3 combinations for brevity
    print(combo)

# params=[
#     {'num_leaves': 511, 'min_data_in_leaf': 100, 'max_depth': 11, 'learning_rate': 0.01, 'lambda_l2': 0.025, 'lambda_l1': 0.01, 'feature_fraction': 0.4},
    # {'num_leaves': 1023, 'min_data_in_leaf': 100, 'max_depth': 15, 'learning_rate': 0.01, 'lambda_l2': 0.5, 'lambda_l1': 0.25, 'feature_fraction': 0.6}
#        ]

seeds=np.random.default_rng().integers(0,1000,nb_seeds)

print(f"{seeds=}")
# loop over the parameters
for param in combinations:
    try:
        for seed in seeds:
            print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} Try : {param}")
            clf=lgb.LGBMClassifier(objective='multiclass',metric='custom',device_type=device_type,verbosity=-1,
                                seed=seed, num_class=nb_class,n_estimators=1000,#np.random.default_rng().integers(0,10000),
                                **param)
            
            for train_index, val_index in scv.split(df_x_train_r, col_y_train):
                X_train, X_val = df_x_train_r.iloc[train_index], df_x_train_r.iloc[val_index]
                y_train, y_val = col_y_train.iloc[train_index], col_y_train.iloc[val_index]

                # train the model with the best parameters and eval set without log
                clf.fit(X_train, y_train,eval_set=[(X_val, y_val)],eval_metric=lgbm_mngr.lgbm_custom_penalty_score,
                        callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric

                print(f"{dt.now().strftime('%Y-%m-%d %H:%M:%S')} Best iteration: {clf.best_iteration_}")

            y_pred = clf.predict(df_x_train_r)
            accuracy_train = accuracy_score(col_y_train, y_pred)
            penalty_score=lgbm_mngr.lgbm_custom_penalty_score(col_y_train, y_pred)
            # print(f'Accuracy Train: {accuracy_train} Penalty Train: {penalty_score}')

            y_pred = clf.predict(df_x_val_r)
            accuracy_val = accuracy_score(col_y_val, y_pred)
            penalty_score=lgbm_mngr.lgbm_custom_penalty_score(col_y_val, y_pred)
            # print(f'Accuracy Val: {accuracy_val} Penalty Val: {penalty_score}')
            
            # Extract precision values
            precision_vals = precision_score(col_y_val, y_pred, average=None)
            precision_str ='precision:'+'-'.join([f'{i}:{precision_vals[i]:.2f}' for i in range(len(precision_vals))])
            
            # Extract recall values
            recall_vals = recall_score(col_y_val, y_pred, average=None)
            recall_str = 'recall:'+'-'.join([f'{i}:{recall_vals[i]:.2f}' for i in range(len(recall_vals))])

            # print(classification_report(col_y_val, y_pred))
            # print the confusion matrix
            print(metrics.confusion_matrix(col_y_val, y_pred))

            # Calculate the depth of each tree
            booster = clf.booster_
            tree_info = booster.dump_model()["tree_info"]
            # print(tree_info)
            max_depths = [lgbm_mngr.calculate_tree_depth(tree["tree_structure"]) for tree in tree_info]
            avg_max_depth = sum(max_depths) / len(max_depths)
            max_max_depth = max(max_depths)
            # print(f"Max depth of trees: {avg_max_depth=} {max_max_depth=}")


            # save the values of param and accuracy_val in a dataframe
            results_df = pd.concat([results_df, 
                                    pd.DataFrame([{'params': param,'accuracy_train':round(accuracy_train,3),
                                                    'accuracy_val': round(accuracy_val,3),'penalty_score':round(penalty_score[1],3),
                                                    'precision':precision_str,'recall':recall_str,'best_iter':clf.best_iteration_,
                                                    'seed':seed,'max_max_depth':max_max_depth,'avg_max_depth':avg_max_depth}])],
                                                    ignore_index=True)
    except Exception as e:
        print(f"Error with parameters: {param}")
        print(f"Error message: {e}")   

    

# save tje results in a file
results_df.to_csv(PATH_DATA_DTS+dts_name+"_50D_LGBM_RESULTS_CLASS_5_V5A.csv", sep=",")
# save the model

# joblib.dump(clf, PATH_DATA_DTS+dts_name+"_LGBM_MODEL_R2.pkl")


In [ ]:
# results_df.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_RESULTS_CLASS_10_V5_a.csv", sep=",")

In [6]:
#  TEST 1 SET OF PARAMETERS for classification datasets !!!!!!!!!
device_type='gpu'
param={'learning_rate': 0.0001, 'max_depth': 20, 'num_leaves': 23, 'min_data_in_leaf': 25, 'feature_fraction': 0.25, 'lambda_l1': 0, 'lambda_l2': 1}
seed=43
nb_class=5
n_splits=3
method='strat'

if method=='time':
    scv = TimeSeriesSplit(n_splits=1)
elif method=='strat':
    scv = StratifiedKFold(n_splits=n_splits, shuffle=False)
else:
    raise ValueError(f"Unknown method: {method}")

clf=lgb.LGBMClassifier(objective='multiclass',metric='custom',device_type=device_type,verbosity=-1,
                                seed=seed, num_class=nb_class,n_estimators=1000,#np.random.default_rng().integers(0,10000),
                                **param)

for train_index, val_index in scv.split(df_x_train_r, col_y_train):
                X_train, X_val = df_x_train_r.iloc[train_index], df_x_train_r.iloc[val_index]
                y_train, y_val = col_y_train.iloc[train_index], col_y_train.iloc[val_index]

                # train the model with the best parameters and eval set without log
                clf.fit(X_train, y_train,eval_set=[(X_val, y_val)],eval_metric=lgbm_mngr.lgbm_custom_penalty_score,
                        callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric
# clf.fit(df_x_train_r, col_y_train, eval_set=[(df_x_val_r, col_y_val)],eval_metric=custom_penalty_score,
#         callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric

y_pred = clf.predict(df_x_train_r)
accuracy_train = accuracy_score(col_y_train, y_pred)
penalty_score=lgbm_mngr.lgbm_custom_penalty_score(col_y_train, y_pred)
# print(f'Accuracy Train: {accuracy_train} Penalty Train: {penalty_score}')

y_pred = clf.predict(df_x_val_r)
accuracy_val = accuracy_score(col_y_val, y_pred)
penalty_score=lgbm_mngr.lgbm_custom_penalty_score(col_y_val, y_pred)
print(f'Accuracy Val: {accuracy_val} Penalty Val: {penalty_score}')

# Extract precision values
precision_vals = precision_score(col_y_val, y_pred, average=None)
precision_str ='precision:'+'-'.join([f'{i}:{precision_vals[i]:.2f}' for i in range(len(precision_vals))])
print(precision_str)

# Extract recall values
recall_vals = recall_score(col_y_val, y_pred, average=None)
recall_str = 'recall:'+'-'.join([f'{i}:{recall_vals[i]:.2f}' for i in range(len(recall_vals))])
print(recall_str)

print(metrics.confusion_matrix(col_y_val, y_pred))

# Calculate the depth of each tree
booster = clf.booster_
tree_info = booster.dump_model()["tree_info"]
# print(tree_info)
max_depths = [lgbm_mngr.calculate_tree_depth(tree["tree_structure"]) for tree in tree_info]
avg_max_depth = sum(max_depths) / len(max_depths)
max_max_depth = max(max_depths)
print(f"Max depth of trees: {avg_max_depth=} {max_max_depth=}")



Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[31]	valid_0's custom_penalty: 3.5896
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[63]	valid_0's custom_penalty: 3.64132
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[50]	valid_0's custom_penalty: 3.61282
Accuracy Val: 0.2560225669957687 Penalty Val: ('custom_penalty', 3.7379548660084625, False)
precision:0:0.25-1:0.23-2:0.26-3:0.22-4:0.35
recall:0:0.31-1:0.21-2:0.34-3:0.20-4:0.23
[[4356 2369 3052 2302 2101]
 [2871 2909 4473 2468 1459]
 [2627 2925 4871 2536 1221]
 [2954 2818 4278 2767 1363]
 [4701 1695 2135 2400 3249]]
Max depth of trees: avg_max_depth=8.86 max_max_depth=14


In [7]:
# calculate the mean and std of col_y_train and col_y_val
print(f"{col_y_train.mean()} {col_y_train.std()}")
print(f"{col_y_val.mean()} {col_y_val.std()}")

2.0 1.4142151373991836
2.0 1.4142235357759336


In [ ]:
#  SAVE THE MODEL

model_name=dts_name+"_50D_LGBM_MODEL_CLAS5_R3_2508_V1"
joblib.dump(clf, PATH_DATA_DTS+model_name+".pkl")

#  Save the metadata of the model in a file : all the params and the results
metadata = {'params': param,'accuracy_train':round(accuracy_train,3),
            'accuracy_val': round(accuracy_val,3),'penalty_score':round(penalty_score[1],3),
            'precision':precision_str,'recall':recall_str,'best_iter':clf.best_iteration_,
            'seed':seed,'max_max_depth':max_max_depth,'avg_max_depth':avg_max_depth}
#  in metadata, add the list of features and label
metadata['features'] = df_x_train_r.columns
metadata['label'] = label

# save the metadata in a file
with open(PATH_DATA_DTS+model_name+".txt", "w") as f:
    for key in metadata:
        f.write(f"{key} : {metadata[key]}\n")


In [ ]:
#  print the feature importance
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,df_x_train_r.columns)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

# save the feature importance in a file
feature_imp.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_FEATURES_D.csv", sep=",")


In [ ]:
# print la map of the model
lgb.plot_importance(clf,importance_type='split',max_num_features=10,figsize=(20,10),title='Feature importance(split)')
lgb.plot_importance(clf,importance_type='gain',max_num_features=10,figsize=(20,10),title='Feature importance(gain)')
plt.show()


In [11]:
y_pred = clf.predict(df_x_train_r)
accuracy = accuracy_score(col_y_train, y_pred)
print(f'Accuracy Train: {accuracy}')


y_pred = clf.predict(df_x_val_r)
accuracy = accuracy_score(col_y_val, y_pred)
print(f'Accuracy Val: {accuracy}')
print(classification_report(col_y_val, y_pred))
# print the confusion matrix
print(metrics.confusion_matrix(col_y_val, y_pred))

Accuracy Train: 0.2919605768695362
Accuracy Val: 0.2639246467817896
              precision    recall  f1-score   support

         0.0       0.27      0.29      0.28     15925
         1.0       0.23      0.14      0.18     15925
         2.0       0.28      0.37      0.32     15925
         3.0       0.23      0.26      0.24     15925
         4.0       0.31      0.25      0.28     15925

    accuracy                           0.26     79625
   macro avg       0.26      0.26      0.26     79625
weighted avg       0.26      0.26      0.26     79625

[[4676 1795 2938 3344 3172]
 [2797 2304 5108 3711 2005]
 [2404 2112 5891 3767 1751]
 [2763 2175 4805 4122 2060]
 [4819 1474 2430 3180 4022]]


In [ ]:

lgb.plot_importance(clf,max_num_features=40, figsize=(7,6), title="LightGBM Feature Importance")
plt.show()
# save features importance in a file
feature_imp=pd.DataFrame()
feature_imp['features']=df_x_train_r.columns
feature_imp['importance']=clf.feature_importances_
feature_imp.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_FEATURE_IMPORTANCE.csv", sep=",")


In [11]:
# load the model PARIS_TREND_1D_20D_V3_LGBM_MODEL_R2.pkl and print the characteristics
clf = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_class_R3_CLIP_LGBM_MODEL_CLAS5_R3_2503_V1.pkl")
print(clf)


LGBMClassifier(device_type='gpu', feature_fraction=0.25, lambda_l1=0,
               lambda_l2=0.1, learning_rate=0.0005, max_depth=25,
               metric='custom', min_data_in_leaf=100, n_estimators=1000,
               num_class=5, num_leaves=63, objective='multiclass', seed=323,
               verbosity=-1)


PREPARATION FOR BACKTEST

In [ ]:
# Load datasets before training
dts_name="PARIS_TREND_1D_V5B_class_R3_CLIP"
PATH_DATA_MODEL = "G:/Python/Data/models/LGBM/"

df_train = joblib.load(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_val = joblib.load(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_conf = joblib.load(PATH_DATA_DTS+dts_name+"_CONF.pkl")

print(f"{df_train.shape=} {df_val.shape=} {df_conf.shape=}")

df_train.shape=(449071, 128) df_val.shape=(95313, 128) df_conf.shape=(113630, 128)


In [ ]:
models_info = [
    {
        "name": "LGBM_CLASS_5_50D",
        "filename": "PARIS_TREND_1D_V5B_class_R3_CLIP_50D_LGBM_MODEL_CLAS5_R3_2508_V1.pkl",
        "label": "lab_perf_50d_class5"
    },
    {
        "name": "LGBM_CLASS_10_50D",
        "filename": "PARIS_TREND_1D_V5B_class_R3_CLIP_50D_LGBM_MODEL_CLAS10_R3_2507_V1.pkl",
        "label": "lab_perf_50d_class10"
    },
    {
        "name": "LGBM_CLASS_5_20D",
        "filename": "PARIS_TREND_1D_V5B_class_R3_CLIP_20D_LGBM_MODEL_CLAS5_R3_2507_V1.pkl",
        "label": "lab_perf_20d_class5"
    },
    {
        "name": "LGBM_CLASS_10_20D",
        "filename": "PARIS_TREND_1D_V5B_class_R3_CLIP_20D_LGBM_MODEL_CLAS10_R3_2507_V1.pkl",
        "label": "lab_perf_20d_class10"
    }

]

list_rem=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME',
    'lab_perf_20d','lab_perf_50d','lab_perf_10d','lab_perf_5d', 'lab_perf_5d','CODE',
    'lab_perf_5d_class3',
    'lab_perf_5d_class5',
    'lab_perf_5d_class10',
    'lab_perf_10d_class3',
    'lab_perf_10d_class5',
    'lab_perf_10d_class10',
    'lab_perf_20d_class3',
    'lab_perf_20d_class5',
    'lab_perf_20d_class10',
    'lab_perf_50d_class3',
    'lab_perf_50d_class5',
    'lab_perf_50d_class10']

# list_keep=['pos_bot_200','pos_ema50_200','adx14','macd_dif','pos_sma50_200','pos_top_200','adx14_neg'
# ,'pos_bot50','adx14_pos','perf_sma200_10d','macd_fast','pos_stdev20_sma20','pos_bot50_lag_10d','perf_sma50_10d'
# ,'sma20_rsi14','pos_sma20_200','perf_adx14_5d','macd_slow','perf_atr14_5d','pos_sma200','pos_sma20_50','adx14_dif'
# ,'pos_bot20_lag_20d','perf_sma20_10d','pos_top50','pos_avg_vol50','stoch14_signal','sma5_rsi14','pos_rsi14_sma5_20'
# ,'pos_sma10_200','pos_top50_lag_10d','pos_bb20_hi_lag_5d','pos_top20_lag_20d','pos_sma5_200','pos_bot20'
# ,'stoch14','pos_bb20_lo_lag_5d','pos_stdev20_sma5','trix12','aroon14_dif','pos_bb20_hi','rsi14','perf_adx14_neg_5d'
# ,'tr_atr14','williamsr_14','pos_bb20_lo','pos_sma50','perf_adx14_pos_5d','perf_stoch14_dif_5d','ret_20d'
# ,'pos_bot20_lag_5d','pos_sma10_50','pos_sma5_50','stdev20_1d','aroon14_dif_lag_5d','pos_top20','perf_sma200_5d'
# ,'perf_sma_200_5d','perf_sma50_5d']

In [5]:
# list of columns starting by 'lab_perf'
list_lab_perf = [col for col in df_conf.columns if col.startswith('lab_perf')]
print(f"List of label columns: {list_lab_perf}")

List of label columns: ['lab_perf_20d', 'lab_perf_50d', 'lab_perf_10d', 'lab_perf_5d', 'lab_perf_5d_class3', 'lab_perf_5d_class5', 'lab_perf_5d_class10', 'lab_perf_10d_class3', 'lab_perf_10d_class5', 'lab_perf_10d_class10', 'lab_perf_20d_class3', 'lab_perf_20d_class5', 'lab_perf_20d_class10', 'lab_perf_50d_class3', 'lab_perf_50d_class5', 'lab_perf_50d_class10']


In [13]:
PATH_DATA_MODEL = "G:/Python/Data/models/LGBM/"
models={}

dfs_x_train={}
cols_y_train={}
dfs_x_val={}
cols_y_val={}
dfs_x_conf={}
cols_y_conf={}

for model in models_info:
    model_name=model['name']
    label=model['label']
    model_filename=model['filename']
    print(f"Model: {model_name},Label: {label},Filename: {model_filename}")

    models[model_name] = joblib.load(PATH_DATA_MODEL+model_filename)
    print(models[model_name])

    df_train_w=df_train.copy()
    df_val_w=df_val.copy()
    df_conf_w=df_conf.copy()

    list_rem_w=list_rem.copy()
    list_rem_w.remove(label)  # remove the label from the list_rem

    # remove columns from df_x_train_r don't crash if not in list
    df_train_w.drop(columns=list_rem_w, inplace=True, errors='ignore')
    df_val_w.drop(columns=list_rem_w, inplace=True, errors='ignore')
    df_conf_w.drop(columns=list_rem_w, inplace=True, errors='ignore')

    print(f"{df_train_w.shape} {df_val_w.shape} {df_conf_w.shape}")

    list_feat=df_train_w.columns.values.tolist()
    list_feat.remove(label)

    dfs_x_train[model_name], cols_y_train[model_name] = sm.split_df_x_y(
        df_in=df_train_w, list_features=list_feat, str_label=label, drop_na=False)

    dfs_x_val[model_name], cols_y_val[model_name] = sm.split_df_x_y(
        df_in=df_val_w, list_features=list_feat, str_label=label, drop_na=False)

    dfs_x_conf[model_name], cols_y_conf[model_name] = sm.split_df_x_y(
        df_in=df_conf_w, list_features=list_feat, str_label=label, drop_na=False)

    list_keep=models[model_name].feature_names_in_

    # remove columns from df_x_train_r idf not in list_keep, don't crash if not in list
    dfs_x_train[model_name].drop(columns=[col for col in dfs_x_train[model_name].columns if col not in list_keep], inplace=True, errors='ignore')
    dfs_x_val[model_name].drop(columns=[col for col in dfs_x_val[model_name].columns if col not in list_keep], inplace=True, errors='ignore')
    dfs_x_conf[model_name].drop(columns=[col for col in dfs_x_conf[model_name].columns if col not in list_keep], inplace=True, errors='ignore')

    print(f"{dfs_x_train[model_name].shape=} {dfs_x_val[model_name].shape=} {dfs_x_conf[model_name].shape=} {cols_y_train[model_name].size=} {cols_y_val[model_name].size=} {cols_y_conf[model_name].size=}")

Model: LGBM_CLASS_5_50D,Label: lab_perf_50d_class5,Filename: PARIS_TREND_1D_V5B_class_R3_CLIP_50D_LGBM_MODEL_CLAS5_R3_2508_V1.pkl
LGBMClassifier(device_type='gpu', feature_fraction=0.25, lambda_l1=0,
               lambda_l2=1, learning_rate=0.0001, max_depth=20, metric='custom',
               min_data_in_leaf=25, n_estimators=1000, num_class=5,
               num_leaves=23, objective='multiclass', seed=43, verbosity=-1)
(449071, 114) (95313, 114) (113630, 114)
dfs_x_train[model_name].shape=(449071, 59) dfs_x_val[model_name].shape=(95313, 59) dfs_x_conf[model_name].shape=(113630, 59) cols_y_train[model_name].size=449071 cols_y_val[model_name].size=95313 cols_y_conf[model_name].size=113630
Model: LGBM_CLASS_10_50D,Label: lab_perf_50d_class10,Filename: PARIS_TREND_1D_V5B_class_R3_CLIP_50D_LGBM_MODEL_CLAS10_R3_2507_V1.pkl
LGBMClassifier(device_type='gpu', feature_fraction=0.25, lambda_l1=0,
               lambda_l2=0.1, learning_rate=0.0001, max_depth=20,
               metric='custom', 

In [14]:
# Check if the indices are aligned
for model in models_info:
    model_name=model['name']
    assert df_train.index.equals(dfs_x_train[model_name].index), "Row order is not consistent!"
    assert df_val.index.equals(dfs_x_val[model_name].index), "Row order is not consistent!"
    assert df_conf.index.equals(dfs_x_conf[model_name].index), "Row order is not consistent!"


In [15]:
# predict the values of the  models

print(f"{df_train.shape=} {df_val.shape=} {df_conf.shape=}")

for model in models_info:
    
    model_name=model['name']
    print(f"Predicting with model: {model_name}")

    df_train['predict_'+model_name]=models[model_name].predict(dfs_x_train[model_name])
    df_val['predict_'+model_name]=models[model_name].predict(dfs_x_val[model_name])
    df_conf['predict_'+model_name]=models[model_name].predict(dfs_x_conf[model_name])
    df_train['predict_'+model_name+'_proba']=models[model_name].predict_proba(dfs_x_train[model_name]).tolist()
    df_val['predict_'+model_name+'_proba']=models[model_name].predict_proba(dfs_x_val[model_name]).tolist()
    df_conf['predict_'+model_name+'_proba']=models[model_name].predict_proba(dfs_x_conf[model_name]).tolist()


print(f"{df_train.shape=} {df_val.shape=} {df_conf.shape=}")

df_train.shape=(449071, 134) df_val.shape=(95313, 134) df_conf.shape=(113630, 134)
Predicting with model: LGBM_CLASS_5_50D
Predicting with model: LGBM_CLASS_10_50D
Predicting with model: LGBM_CLASS_5_20D
Predicting with model: LGBM_CLASS_10_20D
df_train.shape=(449071, 136) df_val.shape=(95313, 136) df_conf.shape=(113630, 136)


In [17]:
df_train['PART']='TRAIN'
df_val['PART']='VAL'
df_conf['PART']='CONF'

df_all=pd.concat([df_train,df_val,df_conf],axis=0)
print(f"{df_all.shape=}")
# save the df_all in a file
joblib.dump(df_all, PATH_DATA_DTS+"PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL.pkl")

df_all.shape=(658014, 137)


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL.pkl']

In [5]:
# print list of columns and index of df_all in a file
with open(PATH_DATA_DTS+"PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL.txt", "w") as f:
    f.write(f"Columns:\n{list(df_all.columns)}\n")
    f.write(f"Index:\n{list(df_all.index)}\n")

In [4]:
df_all=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL.pkl")
print(f"{df_all.shape=}")

df_all.shape=(658014, 137)


In [8]:
df_all_ana=df_all.copy()
col_keep=['pos_sma20_200','rsi14','PART','lab_perf_20d', 'lab_perf_50d',
          'lab_perf_20d_class5', 'lab_perf_20d_class10','lab_perf_50d_class5', 'lab_perf_50d_class10',
          'predict_LGBM_CLASS_5_50D', 'predict_LGBM_CLASS_5_50D_proba', 
'predict_LGBM_CLASS_10_50D', 'predict_LGBM_CLASS_10_50D_proba', 'predict_LGBM_CLASS_5_20D', 'predict_LGBM_CLASS_5_20D_proba',
 'predict_LGBM_CLASS_10_20D', 'predict_LGBM_CLASS_10_20D_proba'
          ]
df_all_ana.drop(columns=[col for col in df_all_ana.columns if col not in col_keep], inplace=True, errors='ignore')
print(f"{df_all_ana.shape=}")

df_all_ana.shape=(658014, 17)


In [9]:
# get stock info with get_info_all_stock

df_stock_infos=sio.get_info_all_stock(con_fwk )
print(f"{df_stock_infos.shape=}")
print(df_stock_infos.head())

df_stock_infos.shape=(403, 8)
   SK_SYMBOL     CODE                    NAME   TYPE  REGION CODE_YAHOO  \
0         12   2MX.PA             2MX ORGANIC  Stock  France     2MX.PA   
1         13   AAA.PA  ALAN ALLMAN ASSOCIATES  Stock  France     AAA.PA   
2         14  ABCA.PA           ABC ARBITRAGE  Stock  France    ABCA.PA   
3         15  ABEO.PA                    ABEO  Stock  France    ABEO.PA   
4         16  ABIO.PA                 ALBIOMA  Stock  France    ABIO.PA   

   TRADABLE SHARESOUTSTANDING  
0         1              None  
1         0          45675754  
2         1          59608879  
3         1           7543305  
4         1          32370737  


In [10]:
# Reset the index of df_all to make 'code' a regular column
df_all_reset = df_all.reset_index()

# Merge df_all with df_stock_infos on the 'code' column
df_all_merged = df_all_reset.merge(df_stock_infos[['CODE', 'SHARESOUTSTANDING', 'NAME', 'TRADABLE']], on='CODE', how='left')

# Restore the multi-index (datetime and code)
df_all_2 = df_all_merged.set_index(['OPEN_DATETIME', 'CODE'])

# Print the updated DataFrame
print(df_all_2.head())

                      OPEN   HIGH    LOW  CLOSE  VOLUME  pos_sma20  pos_sma50  \
OPEN_DATETIME CODE                                                              
2011-06-08    AB.PA  10.77  10.95  10.01  10.12  182867      0.083      0.096   
2011-06-09    AB.PA  10.15  10.27   9.74  10.04  132862      0.066      0.086   
2011-06-10    AB.PA  10.11  10.14   9.91  10.00   51809      0.055      0.080   
2011-06-13    AB.PA  10.15  10.65  10.08  10.60  211391      0.110      0.142   
2011-06-14    AB.PA  10.62  11.23  10.45  10.66  302799      0.108      0.145   

                     pos_sma200  pos_sma50_200  pos_sma20_50  ...  \
OPEN_DATETIME CODE                                            ...   
2011-06-08    AB.PA       0.085         -0.010         0.013  ...   
2011-06-09    AB.PA       0.078         -0.008         0.019  ...   
2011-06-10    AB.PA       0.074         -0.006         0.024  ...   
2011-06-13    AB.PA       0.139         -0.003         0.029  ...   
2011-06-14    AB.P

In [11]:
#check type of f_all_2['CLOSE'] and df_all_2['SHARESOUTSTANDING']
print(f"{df_all_2['CLOSE'].dtype=} {df_all_2['SHARESOUTSTANDING'].dtype=}")

df_all_2['CLOSE'].dtype=dtype('float64') df_all_2['SHARESOUTSTANDING'].dtype=dtype('O')


In [12]:
from numpy import int64

df_all_2['NAME']=df_all_2['NAME'].astype(str)
df_all_2['TRADABLE']=df_all_2['TRADABLE'].astype(int)
df_all_2['SHARESOUTSTANDING']=df_all_2['SHARESOUTSTANDING'].fillna(0)
df_all_2['SHARESOUTSTANDING']=df_all_2['SHARESOUTSTANDING'].replace('',0)
df_all_2['SHARESOUTSTANDING']=df_all_2['SHARESOUTSTANDING'].astype(int64)
df_all_2['cap_M']=round((df_all_2['CLOSE']*df_all_2['SHARESOUTSTANDING'])/1000000.0,2)
df_all_2.head(5)

,,OPEN,HIGH,LOW,CLOSE,VOLUME,pos_sma20,pos_sma50,pos_sma200,pos_sma50_200,pos_sma20_50,...,predict_LGBM_CLASS_10_50D_proba,predict_LGBM_CLASS_5_20D,predict_LGBM_CLASS_5_20D_proba,predict_LGBM_CLASS_10_20D,predict_LGBM_CLASS_10_20D_proba,PART,SHARESOUTSTANDING,NAME,TRADABLE,cap_M
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
2011-06-08,AB.PA,10.77,10.95,10.01,10.12,182867,0.083,0.096,0.085,-0.010,0.013,...,"[0.10047941089122911, 0.09999395204666037, 0.0...",0.0,"[0.2002122949914179, 0.19990382602724743, 0.19...",0.0,"[0.10029605371291615, 0.0999946796758388, 0.09...",TRAIN,51071069,AB SCIENCE,1,516.84
2011-06-09,AB.PA,10.15,10.27,9.74,10.04,132862,0.066,0.086,0.078,-0.008,0.019,...,"[0.10051213533097957, 0.10000555566192722, 0.0...",0.0,"[0.20020875874149463, 0.19988977179184939, 0.1...",9.0,"[0.10022380610395494, 0.09999952968378305, 0.0...",TRAIN,51071069,AB SCIENCE,1,512.75
2011-06-10,AB.PA,10.11,10.14,9.91,10.00,51809,0.055,0.080,0.074,-0.006,0.024,...,"[0.10049750650006703, 0.09999898725380174, 0.0...",0.0,"[0.20016980268082513, 0.1998968832693471, 0.19...",0.0,"[0.1002193259484751, 0.09998635563499617, 0.09...",TRAIN,51071069,AB SCIENCE,1,510.71
2011-06-13,AB.PA,10.15,10.65,10.08,10.60,211391,0.110,0.142,0.139,-0.003,0.029,...,"[0.10049522063394069, 0.10000754434352764, 0.0...",0.0,"[0.20018682265913199, 0.19991753990649885, 0.1...",0.0,"[0.10023083589438259, 0.09999512886467173, 0.0...",TRAIN,51071069,AB SCIENCE,1,541.35
2011-06-14,AB.PA,10.62,11.23,10.45,10.66,302799,0.108,0.145,0.147,0.001,0.034,...,"[0.10049224722561091, 0.10003109876198084, 0.0...",0.0,"[0.2001546685123644, 0.1999382800172052, 0.199...",9.0,"[0.10023833421539097, 0.09999440595911645, 0.0...",TRAIN,51071069,AB SCIENCE,1,544.42


In [ ]:
# print(df_all_2.shape)
# joblib.dump(df_all_2, PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL2.pkl")

# with open(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL_INFO2.txt", "w") as f:
#     f.write(f"Columns:\n{list(df_all_2.columns)}\n")
#     f.write(f"Index:\n{list(df_all_2.index.levels)}\n")


(671570, 136)


In [ ]:
# df_all_2=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL2.pkl")
# print(f"{df_all_2.shape=}")

df_all_2.shape=(671570, 136)


In [13]:
#  load BASE eand merge it with df_all_2
# df_base = pd.read_csv(PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
df_base = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3B_BASE.pkl")
print(f"{df_base.shape=}")

df_base.shape=(1299247, 160)


In [14]:
# check and cast index of the df

df_base_reset = df_base.reset_index()
df_all_2_reset = df_all_2.reset_index()

# Cast types
df_base_reset['OPEN_DATETIME'] = pd.to_datetime(df_base_reset['OPEN_DATETIME'])
df_base_reset['CODE'] = df_base_reset['CODE'].astype(str)
df_all_2_reset['OPEN_DATETIME'] = pd.to_datetime(df_all_2_reset['OPEN_DATETIME'])
df_all_2_reset['CODE'] = df_all_2_reset['CODE'].astype(str)

# Set index back
df_base = df_base_reset.set_index(['OPEN_DATETIME', 'CODE'])
df_all_2 = df_all_2_reset.set_index(['OPEN_DATETIME', 'CODE'])


print("df_base OPEN_DATETIME dtype:", df_base.index.get_level_values('OPEN_DATETIME').dtype)
print("df_base CODE dtype:", df_base.index.get_level_values('CODE').dtype)
print("df_all_2 OPEN_DATETIME dtype:", df_all_2.index.get_level_values('OPEN_DATETIME').dtype)
print("df_all_2 CODE dtype:", df_all_2.index.get_level_values('CODE').dtype)


df_base OPEN_DATETIME dtype: datetime64[ns]
df_base CODE dtype: object
df_all_2 OPEN_DATETIME dtype: datetime64[ns]
df_all_2 CODE dtype: object


In [15]:
# do a left join on df_base and df_all_2 on the index (OPEN_DATETIME, CODE)
# keep all the columns of df_base and df_all_2
df_all_3 = df_base.join(df_all_2, how='left', on=['OPEN_DATETIME','CODE'],  rsuffix='_val',)
print(f"{df_all_3.shape=}")

df_all_3.shape=(1299247, 301)


In [16]:
df_merged=df_all_3.copy()

#  TODO choose the columns !!!!!
df_merged=df_all_3[ ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME','pos_sma20_200','adx14_pos',
                    'rsi14','PART','lab_perf_20d', 'lab_perf_50d',
          'lab_perf_20d_class5', 'lab_perf_20d_class10','lab_perf_50d_class5', 'lab_perf_50d_class10',
          'predict_LGBM_CLASS_5_50D', 'predict_LGBM_CLASS_5_50D_proba', 
'predict_LGBM_CLASS_10_50D', 'predict_LGBM_CLASS_10_50D_proba', 'predict_LGBM_CLASS_5_20D', 'predict_LGBM_CLASS_5_20D_proba',
 'predict_LGBM_CLASS_10_20D', 'predict_LGBM_CLASS_10_20D_proba', 'SHARESOUTSTANDING', 'NAME', 'TRADABLE', 'cap_M']]

print(df_merged.shape)

(1299247, 27)


In [17]:
# delete objects from memory df_base, df_all, all2,df_all3
del df_base, df_all, df_all_2, df_all_3


In [18]:
df_merged.head()

,,OPEN,HIGH,LOW,CLOSE,VOLUME,pos_sma20_200,adx14_pos,rsi14,PART,lab_perf_20d,...,predict_LGBM_CLASS_10_50D,predict_LGBM_CLASS_10_50D_proba,predict_LGBM_CLASS_5_20D,predict_LGBM_CLASS_5_20D_proba,predict_LGBM_CLASS_10_20D,predict_LGBM_CLASS_10_20D_proba,SHARESOUTSTANDING,NAME,TRADABLE,cap_M
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
2010-04-26,AB.PA,12.98,12.98,12.20,12.68,62866,NaN,NaN,NaN,NaN,-0.064669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-27,AB.PA,12.74,12.83,12.61,12.70,22370,NaN,NaN,NaN,NaN,-0.159055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-28,AB.PA,12.70,12.70,12.41,12.50,8211,NaN,NaN,NaN,NaN,-0.072800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-29,AB.PA,12.60,12.65,12.46,12.64,4676,NaN,NaN,NaN,NaN,-0.058544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,AB.PA,12.63,12.71,12.55,12.65,4470,NaN,NaN,NaN,NaN,-0.039526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# save df_merged in a file
# joblib.dump(df_merged, PATH_DATA_DTS+"PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL3.pkl")

['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL3.pkl']

In [ ]:
# df_merged=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL3.pkl")
# print(f"{df_merged.shape=}")

df_merged.shape=(1299247, 18)


In [5]:
# print min and max of df_merged.index.get_level_values('OPEN_DATETIME') group by df_merged['PART']
df_merged_reset = df_merged.reset_index()  # Reset the index to access 'OPEN_DATETIME' as a column
print(df_merged_reset.groupby('PART')['OPEN_DATETIME'].agg(['min', 'max']))
# print nb line where df_merged['PART']is null
print(f"Nb line where PART is null: {df_merged[df_merged['PART'].isnull()].shape[0]}")

             min        max
PART                       
CONF  2020-10-01 2023-11-01
TRAIN 1991-10-23 2017-07-31
VAL   2017-08-01 2020-09-30
Nb line where PART is null: 627677


In [19]:
# calculate values for 'PART', 'SHARESOUTSTANDING', 'NAME', 'TRADABLE', 'cap_M'
df_merged['PART'] = df_merged.index.get_level_values('OPEN_DATETIME').map(
    lambda x: 'TRAIN' if x < pd.Timestamp('2017-08-01') else 
              'VAL' if x < pd.Timestamp('2020-10-01') else 
              'CONF' if pd.notnull(x) else None
)
df_merged['NAME'] = df_merged.groupby(df_merged.index.get_level_values('CODE'))['NAME'].transform(lambda x: x.ffill().bfill())
df_merged['SHARESOUTSTANDING'] = df_merged.groupby(df_merged.index.get_level_values('CODE'))['SHARESOUTSTANDING'].transform(lambda x: x.ffill().bfill())
df_merged['TRADABLE'] = df_merged.groupby(df_merged.index.get_level_values('CODE'))['TRADABLE'].transform(lambda x: x.ffill().bfill())
df_merged['cap_M'] = round((df_merged['CLOSE']*df_merged['SHARESOUTSTANDING'])/1000000.0,2)
# drop line where NAME is null
# df_merged.dropna(subset=['NAME'], inplace=True)

In [20]:
print(f"Nb line where PART is null: {df_merged[df_merged['PART'].isnull()].shape[0]}")
print(f"Nb line where NAME is null: {df_merged[df_merged['NAME'].isnull()].shape[0]}")
print(f"Nb line where SHARESOUTSTANDING is null: {df_merged[df_merged['SHARESOUTSTANDING'].isnull()].shape[0]}")
print(f"Nb line where TRADABLE is null: {df_merged[df_merged['TRADABLE'].isnull()].shape[0]}")
print(f"Nb line where cap_M is null: {df_merged[df_merged['cap_M'].isnull()].shape[0]}")
print(f"{df_merged.shape=}")

df_merged.dropna(subset=['NAME'], inplace=True)
print(f"{df_merged.shape=}")

Nb line where PART is null: 0
Nb line where NAME is null: 1440
Nb line where SHARESOUTSTANDING is null: 1440
Nb line where TRADABLE is null: 1440
Nb line where cap_M is null: 1444
df_merged.shape=(1299247, 27)
df_merged.shape=(1297807, 27)


In [21]:
# print for the code AI.PA the different NAME of the dataset 
print(df_merged[df_merged.index.get_level_values('CODE')=='AI.PA']['NAME'].unique())

['AIR LIQUIDE']


In [22]:
joblib.dump(df_merged, PATH_DATA_DTS+"PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL4.pkl")

['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5B_lab_20_50_class_5_10_PREDICT_ALL4.pkl']

In [23]:
# Extract the datetime level from the MultiIndex and apply strftime
df_ai = df_merged[df_merged.index.get_level_values('CODE') == 'AI.PA']
df_ai['YYYYMM'] = df_ai.index.get_level_values('OPEN_DATETIME').strftime('%Y%m')
# Count rows by YYYYMM and sort by YYYYMM
df_ai_count = df_ai.groupby('YYYYMM').size().reset_index(name='count')
# print df_ai for year 2022
print(df_ai_count[df_ai_count['YYYYMM'].str.startswith('2022')])

     YYYYMM  count
264  202201     21
265  202202     20
266  202203     23
267  202204     19
268  202205     22
269  202206     22
270  202207     21
271  202208     23
272  202209     22
273  202210     21
274  202211     22
275  202212     21


C:\Users\Benoit\AppData\Local\Temp\ipykernel_7844\2564425692.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ai['YYYYMM'] = df_ai.index.get_level_values('OPEN_DATETIME').strftime('%Y%m')


Merge adjusted dataset and not adjusted dataset

In [25]:
df_adj=joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_ALL4.pkl")
print(df_adj.shape)
print("df_adj OPEN_DATETIME dtype:", df_adj.index.get_level_values('OPEN_DATETIME').dtype)
print("df_adj CODE dtype:", df_adj.index.get_level_values('CODE').dtype)

(1233792, 18)
df_adj OPEN_DATETIME dtype: datetime64[ns]
df_adj CODE dtype: object


In [26]:
# merge df_adj with df_merged with a full outer join on the 2 columns index
df_merged = df_merged.merge(df_adj, on=['OPEN_DATETIME', 'CODE'], how='outer', suffixes=('', '_adj'))
print(df_merged.shape)

(1297807, 45)


In [27]:
joblib.dump(df_merged, PATH_DATA_DTS+"PARIS_TREND_1D_V5_5B_lab_20_adj_20_50_class_5_10_PREDICT_FULL.pkl")

['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_5B_lab_20_adj_20_50_class_5_10_PREDICT_FULL.pkl']

In [3]:
df_full = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_5B_lab_20_adj_20_50_class_5_10_PREDICT_FULL.pkl")
print(df_full.shape)
print(df_full.columns)

(1297807, 45)
Index(['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'pos_sma20_200', 'adx14_pos',
       'rsi14', 'PART', 'lab_perf_20d', 'lab_perf_50d', 'lab_perf_20d_class5',
       'lab_perf_20d_class10', 'lab_perf_50d_class5', 'lab_perf_50d_class10',
       'predict_LGBM_CLASS_5_50D', 'predict_LGBM_CLASS_5_50D_proba',
       'predict_LGBM_CLASS_10_50D', 'predict_LGBM_CLASS_10_50D_proba',
       'predict_LGBM_CLASS_5_20D', 'predict_LGBM_CLASS_5_20D_proba',
       'predict_LGBM_CLASS_10_20D', 'predict_LGBM_CLASS_10_20D_proba',
       'SHARESOUTSTANDING', 'NAME', 'TRADABLE', 'cap_M', 'OPEN_adj',
       'HIGH_adj', 'LOW_adj', 'CLOSE_adj', 'VOLUME_adj', 'pos_sma20_200_adj',
       'adx14_pos_adj', 'lab_perf_20d_class_5', 'lab_perf_20d_class_10',
       'predict_10', 'predict_5', 'predict_10_proba', 'predict_5_proba',
       'PART_adj', 'SHARESOUTSTANDING_adj', 'NAME_adj', 'TRADABLE_adj',
       'cap_M_adj'],
      dtype='object')


In [4]:
# in df_full rename coluns 'lab_perf_20d_class_5', 'lab_perf_20d_class_10', 'predict_10', 'predict_5', 'predict_10_proba', 'predict_5_proba' adding '_adj' suffix
df_full = df_full.rename(columns={
    'lab_perf_20d_class_5': 'lab_perf_20d_class_5_adj',
    'lab_perf_20d_class_10': 'lab_perf_20d_class_10_adj',
    'predict_10': 'predict_10_adj',
    'predict_5': 'predict_5_adj',
    'predict_10_proba': 'predict_10_proba_adj',
    'predict_5_proba': 'predict_5_proba_adj'
})

In [8]:
# print nb lines group by PART and group by PART_ADJ and count nb lines with part<>part_adj
print(df_full['PART'].value_counts())
print(df_full['PART_adj'].value_counts())
print(f"Number of lines where PART != PART_adj: {df_full[df_full['PART'] != df_full['PART_adj']].shape[0]}")

PART
TRAIN    843999
CONF     238684
VAL      215124
Name: count, dtype: int64
PART_adj
TRAIN    798157
CONF     229421
VAL      206214
Name: count, dtype: int64
Number of lines where PART != PART_adj: 64015


In [13]:
# print top 10 lines with PART != PART_adj
# print(df_full[df_full['PART'] != df_full['PART_adj']].head(10))

# print distinct NAME with PART != PART_adj
print(df_full[df_full['PART'] != df_full['PART_adj']]['NAME'].unique())

['CYBERGUN' 'FLEURY MICHON' 'BURELLE' 'CRCAM SUD R.A.CCI'
 'ELEC.STRASBOURG' 'EURO RESSOURCES' 'IMMOB.DASSAULT' 'SOLOCAL GROUP'
 'L.D.C.' 'MAUREL ET PROM' 'SAMSE']


In [19]:
df_part=df_full[df_full['PART'] != df_full['PART_adj']]
print(f"{df_part.shape=}")

df_part[['NAME', 'PART', 'PART_adj']] = df_part[['NAME', 'PART', 'PART_adj']].fillna('NA')
print(df_part.groupby(['NAME', 'PART', 'PART_adj']).size())


df_part.shape=(64015, 45)
NAME               PART   PART_adj
BURELLE            CONF   NA           842
                   TRAIN  NA          4496
                   VAL    NA           810
CRCAM SUD R.A.CCI  CONF   NA           842
                   TRAIN  NA          4520
                   VAL    NA           810
CYBERGUN           CONF   NA           842
                   TRAIN  NA           148
                   VAL    NA           810
ELEC.STRASBOURG    CONF   NA           842
                   TRAIN  NA          6737
                   VAL    NA           810
EURO RESSOURCES    CONF   NA           842
                   TRAIN  NA          3212
                   VAL    NA           810
FLEURY MICHON      CONF   NA           842
                   TRAIN  NA          4468
                   VAL    NA           810
IMMOB.DASSAULT     CONF   NA           842
                   TRAIN  NA          4523
                   VAL    NA           810
L.D.C.             CONF   NA        

C:\Users\Benoit\AppData\Local\Temp\ipykernel_708\3678162494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part[['NAME', 'PART', 'PART_adj']] = df_part[['NAME', 'PART', 'PART_adj']].fillna('NA')


In [20]:
# if df_full[PART_adj] is none PART_adj=PART
df_full['PART_adj'] = df_full['PART_adj'].fillna(df_full['PART'])

In [22]:
print(df_full['predict_LGBM_CLASS_5_20D'].value_counts())
print(df_full['lab_perf_20d_class_5_adj'].value_counts())

# print nb line where predict_LGBM_CLASS_5_20D is not None and lab_perf_20d_class_5_adj is None
print(df_full[df_full['predict_LGBM_CLASS_5_20D'].notna() & df_full['lab_perf_20d_class_5_adj'].isna()].shape[0])
print(df_full[df_full['predict_LGBM_CLASS_5_20D'].isna() & df_full['lab_perf_20d_class_5_adj'].notna()].shape[0])

predict_LGBM_CLASS_5_20D
2.0    149183
3.0    143657
4.0    142654
0.0    141844
1.0     80676
Name: count, dtype: int64
lab_perf_20d_class_5_adj
0.0    146927
1.0    132407
4.0    132225
3.0    131023
2.0    128980
Name: count, dtype: int64
32883
46431


In [24]:
print(f"{df_full.shape=}")
print(df_full[df_full['predict_LGBM_CLASS_5_20D'].notna() & df_full['lab_perf_20d_class_5_adj'].notna()].shape[0] &
      df_full[df_full['predict_LGBM_CLASS_10_20D'].notna() & df_full['lab_perf_20d_class_10_adj'].notna()].shape[0] &
      df_full[df_full['predict_LGBM_CLASS_5_50D'].notna() & df_full['predict_LGBM_CLASS_10_50D'].notna()].shape[0]
      )

df_full.shape=(1297807, 45)
526402


In [27]:
list_scores=['predict_LGBM_CLASS_5_20D', 'predict_LGBM_CLASS_10_20D', 'predict_LGBM_CLASS_5_50D', 'predict_LGBM_CLASS_10_50D','lab_perf_20d_class_5_adj','lab_perf_20d_class_10_adj']
df_full_score = df_full.dropna(subset=list_scores)
print(df_full_score.shape)
print(df_full_score['PART'].value_counts())


(625131, 45)
PART
TRAIN    423488
CONF     110147
VAL       91496
Name: count, dtype: int64


In [28]:
list_rem=['VOLUME_adj','NAME_adj','TRADABLE_adj','PART_adj']
# drop column list_rem of df_full
df_full = df_full.drop(columns=list_rem)
print(f"{df_full.shape=}")

df_full.shape=(1297807, 41)


In [29]:
joblib.dump(df_full, PATH_DATA_DTS+"PARIS_TREND_1D_V5_5B_lab_20_adj_20_50_class_5_10_PREDICT_FULL_V2.pkl")

['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_V5_5B_lab_20_adj_20_50_class_5_10_PREDICT_FULL_V2.pkl']

In [31]:
# df_full.head()
print(df_full['TRADABLE'].value_counts())

TRADABLE
1.0    1297807
Name: count, dtype: int64


Analyse proba

In [23]:
import joblib
import pandas as pd

# Load the DataFrame (adjust path if needed)
PATH_DATA = "G:\\Python\\Data"
PATH_DATA_DTS=PATH_DATA+"\\DTS_FULL\\"
df_all_ana = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_V5_5B_lab_20_adj_20_50_class_5_10_PREDICT_FULL_V2.pkl")

print(df_all_ana.shape)

label_cols_5 = ['lab_perf_50d_class5','lab_perf_20d_class5',  'lab_perf_20d_class_5_adj']
pred_cols_5=['predict_LGBM_CLASS_5_50D','predict_LGBM_CLASS_5_20D','predict_5_adj']
pred_cols_10=['predict_LGBM_CLASS_10_50D','predict_LGBM_CLASS_10_20D','predict_10_adj']

# remove lines where one of pred_cols_5 is NaN
df_all_ana = df_all_ana[df_all_ana[pred_cols_5+pred_cols_10].notna().all(axis=1)]
print(df_all_ana.shape)



(1297807, 41)
(625139, 41)


In [11]:
# print head(5) of columns predict_10_proba_adj, predict_LGBM_CLASS_10_20D_proba, predict_LGBM_CLASS_10_50D_proba
# where the col predict_10_proba_adj is not Nan
print(df_all_ana[df_all_ana['predict_10_proba_adj'].notna()][['predict_10_proba_adj', 'predict_LGBM_CLASS_10_20D_proba', 'predict_LGBM_CLASS_10_50D_proba']].head(5))

                                                  predict_10_proba_adj  \
OPEN_DATETIME CODE                                                       
2011-06-08    AB.PA  [0.10063522928819303, 0.10003987190076463, 0.0...   
2011-06-09    AB.PA  [0.10056879038068549, 0.10001784426967043, 0.0...   
2011-06-10    AB.PA  [0.1005913314236855, 0.10005092628201277, 0.09...   
2011-06-13    AB.PA  [0.10059644152602594, 0.10000801846000389, 0.0...   
2011-06-14    AB.PA  [0.1005894383320005, 0.09999281630481312, 0.09...   

                                       predict_LGBM_CLASS_10_20D_proba  \
OPEN_DATETIME CODE                                                       
2011-06-08    AB.PA  [0.10029605371291615, 0.0999946796758388, 0.09...   
2011-06-09    AB.PA  [0.10022380610395494, 0.09999952968378305, 0.0...   
2011-06-10    AB.PA  [0.1002193259484751, 0.09998635563499617, 0.09...   
2011-06-13    AB.PA  [0.10023083589438259, 0.09999512886467173, 0.0...   
2011-06-14    AB.PA  [0.1002383342153

In [24]:
import lgbm_mngr

# Expand probabilities for predict_10_proba
df_all_ana = lgbm_mngr.expand_probabilities(df_all_ana, 'predict_10_20d_proba_adj', 'predict_10_proba_adj', 10)
df_all_ana = lgbm_mngr.expand_probabilities(df_all_ana, 'predict_10_20d_proba', 'predict_LGBM_CLASS_10_20D_proba', 10)
df_all_ana = lgbm_mngr.expand_probabilities(df_all_ana, 'predict_10_50d_proba', 'predict_LGBM_CLASS_10_50D_proba', 10)

# Expand probabilities for predict_5_proba
df_all_ana = lgbm_mngr.expand_probabilities(df_all_ana, 'predict_5_20d_proba_adj', 'predict_5_proba_adj', 5)
df_all_ana = lgbm_mngr.expand_probabilities(df_all_ana, 'predict_5_20d_proba', 'predict_LGBM_CLASS_5_20D_proba', 5)
df_all_ana = lgbm_mngr.expand_probabilities(df_all_ana, 'predict_5_50d_proba', 'predict_LGBM_CLASS_5_50D_proba', 5)

df_ana_train=df_all_ana[df_all_ana['PART'].isin(['TRAIN'])].copy()

print(df_ana_train.shape)
print(df_ana_train.head())

(423488, 86)
                      OPEN   HIGH    LOW  CLOSE  VOLUME  pos_sma20_200  \
OPEN_DATETIME CODE                                                       
2011-06-08    AB.PA  10.77  10.95  10.01  10.12  182867       0.002418   
2011-06-09    AB.PA  10.15  10.27   9.74  10.04  132862       0.011029   
2011-06-10    AB.PA  10.11  10.14   9.91  10.00   51809       0.018075   
2011-06-13    AB.PA  10.15  10.65  10.08  10.60  211391       0.026391   
2011-06-14    AB.PA  10.62  11.23  10.45  10.66  302799       0.035048   

                     adx14_pos      rsi14   PART  lab_perf_20d  ...  \
OPEN_DATETIME CODE                                              ...   
2011-06-08    AB.PA  44.610561  61.801620  TRAIN     -0.072134  ...   
2011-06-09    AB.PA  41.359485  60.347610  TRAIN     -0.073705  ...   
2011-06-10    AB.PA  39.997237  59.592665  TRAIN     -0.080000  ...   
2011-06-13    AB.PA  43.501693  66.385585  TRAIN     -0.151887  ...   
2011-06-14    AB.PA  46.752155  66.983328 

In [21]:
import numpy as np
# Find rows where 'predict_10_proba_adj' is not a list/array
mask = ~df_all_ana['predict_10_proba_adj'].apply(lambda x: isinstance(x, (list, np.ndarray)))
problematic_rows = df_all_ana[mask]
print(problematic_rows[['predict_10_proba_adj']])
print(f"Number of problematic rows: {problematic_rows.shape[0]}")

Empty DataFrame
Columns: [predict_10_proba_adj]
Index: []
Number of problematic rows: 0


In [20]:
# print lines for the index OPEN_DATETIME 2023-11-01 and CODE AI.PA
print(df_all_ana.loc[
	(df_all_ana.index.get_level_values('OPEN_DATETIME') == pd.Timestamp('2023-11-01')) &
	(df_all_ana.index.get_level_values('CODE') == 'AI.PA')
])

Empty DataFrame
Columns: [OPEN, HIGH, LOW, CLOSE, VOLUME, pos_sma20_200, adx14_pos, rsi14, PART, lab_perf_20d, lab_perf_50d, lab_perf_20d_class5, lab_perf_20d_class10, lab_perf_50d_class5, lab_perf_50d_class10, predict_LGBM_CLASS_5_50D, predict_LGBM_CLASS_5_50D_proba, predict_LGBM_CLASS_10_50D, predict_LGBM_CLASS_10_50D_proba, predict_LGBM_CLASS_5_20D, predict_LGBM_CLASS_5_20D_proba, predict_LGBM_CLASS_10_20D, predict_LGBM_CLASS_10_20D_proba, SHARESOUTSTANDING, NAME, TRADABLE, cap_M, OPEN_adj, HIGH_adj, LOW_adj, CLOSE_adj, pos_sma20_200_adj, adx14_pos_adj, lab_perf_20d_class_5_adj, lab_perf_20d_class_10_adj, predict_10_adj, predict_5_adj, predict_10_proba_adj, predict_5_proba_adj, SHARESOUTSTANDING_adj, cap_M_adj]
Index: []

[0 rows x 41 columns]


In [14]:
# if predict_10=9 what is the mean of each predict_10_proba_i
# print(df_all_ana[df_all_ana['predict_10']==9].loc[:, 'predict_10_proba_0':'predict_10_proba_9'].mean())

print(df_all_ana[df_all_ana['predict_10']==9].loc[:, 'predict_10_proba_0':'predict_10_proba_9'].mean())

predict_10_proba_0    0.100285
predict_10_proba_1    0.100040
predict_10_proba_2    0.099894
predict_10_proba_3    0.099835
predict_10_proba_4    0.099813
predict_10_proba_5    0.099797
predict_10_proba_6    0.099845
predict_10_proba_7    0.099877
predict_10_proba_8    0.100016
predict_10_proba_9    0.100598
dtype: float64


In [ ]:
# for class 5 and predict_5 in (3,4), create a new df aggregated by PART, predict_5, lab_perf_20d_class_5
# and calculate the count of line, mean of each predict_5_proba_i
limit =0.202 # keep 0.202 instead of 0.2015
df_w=df_all_ana[df_all_ana['predict_5_proba_4']>limit]
df_all_ana_5=df_w[(df_w['predict_5'].isin([3,4])) ].groupby(['PART','predict_5','lab_perf_20d_class_5']).agg(
    count=('predict_5','count'),
    mean_0=('predict_5_proba_0','mean'),
    mean_1=('predict_5_proba_1','mean'),
    mean_2=('predict_5_proba_2','mean'),
    mean_3=('predict_5_proba_3','mean'),
    mean_4=('predict_5_proba_4','mean')
).reset_index()
df_all_ana_5.to_csv(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_5_3.csv", sep=",")

In [ ]:
# for class 10 and predict_10 in (8,9), create a new df aggregated by PART, predict_10, lab_perf_20d_class_10
# and calculate the count of line, mean of each predict_10_proba_i
limit =0.101 # keep 
df_w = df_all_ana[(df_all_ana['predict_10_proba_9'] > limit) | (df_all_ana['predict_10_proba_8'] > limit)]
df_all_ana_10=df_w[(df_w['predict_10'].isin([8,9])) ].groupby(['PART','predict_10','lab_perf_20d_class_10']).agg(
    count=('predict_10','count'),
    mean_0=('predict_10_proba_0','mean'),
    mean_1=('predict_10_proba_1','mean'),
    mean_2=('predict_10_proba_2','mean'),
    mean_3=('predict_10_proba_3','mean'),
    mean_4=('predict_10_proba_4','mean'),
    mean_5=('predict_10_proba_5','mean'),
    mean_6=('predict_10_proba_6','mean'),
    mean_7=('predict_10_proba_7','mean'),
    mean_8=('predict_10_proba_8','mean'),
    mean_9=('predict_10_proba_9','mean')
).reset_index()

df_all_ana_10.to_csv(PATH_DATA_DTS+"PARIS_TREND_1D_V5_lab_20_class_5_10_PREDICT_10_2.csv", sep=",")

In [15]:
df_conf=df_all_ana[df_all_ana['PART']=='CONF']
# df_conf_c5_4=df_conf[df_conf['predict_5']==4]
df_conf_c10_9=df_conf[df_conf['predict_10']==9]


In [ ]:
# print a bar chart from df_conf_c5_4 with a serie by lab_perf_20d_class_5 [0-4] 
# X will the proba in 10 groupsand Y the count of line
# bins = np.linspace(0.199, 0.21, 11)
# df_conf_c5_4['predict_5_proba_4_bins'] = pd.cut(df_conf_c5_4['predict_5_proba_4'], bins=bins, include_lowest=True)
# bar_data = df_conf_c5_4.groupby(['predict_5_proba_4_bins', 'lab_perf_20d_class_5']).size().unstack(1)

bins = np.linspace(0.0995, 0.1035, 11)
df_conf_c10_9['predict_10_proba_9_bins'] = pd.cut(df_conf_c10_9['predict_10_proba_9'], bins=bins, include_lowest=True)
bar_data = df_conf_c10_9.groupby(['predict_10_proba_9_bins', 'lab_perf_20d_class_10']).size().unstack(1)


# Plot the grouped bar chart
bar_data.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Probability Bins')
plt.ylabel('Count')
plt.title('Bar Chart of Predict_C_Proba_X by Lab_Perf_20d_Class_C')
plt.legend(title='lab_perf_20d_class')
plt.show()
